In [ ]:
## IMPORTANT: On Colab, we expect your homework to be in the cs189 folder
## Please contact staff if you encounter any problems with installing dependencies
import sys
IS_COLAB = 'google.colab' in sys.modules
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/cs189/hw/hw4
    %pip install -r ./requirements.txt
    !pip install -U kaleido plotly
    import kaleido
    kaleido.get_chrome_sync()

import plotly.io as pio
pio.renderers.default = pio.renderers.default + "+png"

# Last updated: 2025-11-07 01:25:59

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw4_part1.ipynb")

# Homework 4.1

CNNs and Transformers have *transformed* machine learning, opening up whole new domains of innovation: [object detection for autonomous vehicles](https://neptune.ai/blog/self-driving-cars-with-convolutional-neural-networks-cnn), image generation ([Dall-E](https://openai.com/index/dall-e/)), [LLMs](https://developers.google.com/machine-learning/crash-course/llm/transformers), [protein-sequence modeling](https://deepmind.google/science/alphafold/), and more!

In this homework, we're going to explore CNNs and Transformers by implementing some foundational CNN architectures using `PyTorch` and coding some key transformer building blocks from scratch.

---

## Due Date: Friday, November 21, 11:59 PM

This assignment is due on **Friday, November 21, at 11:59 PM**. You must submit your work to Gradescope by this deadline. Please refer to the syllabus for the [Slip Day policy](https://eecs189.org/fa25/syllabus/#slip-days). No late submissions will be accepted beyond the details outlined in the Slip Day policy.

## Deliverables:
* Homework 4.1 consists of paper questions and coding questions.
* Paper questions:
    * Please write or type your answers to the paper questions and submit your answers as a **PDF** to the `HW 4 Written Questions` assignment on Gradescope.
* Code [PDF]:
    * Please submit a **PDF** of your coding notebook to the `HW 4.1 Coding [PDF]` assignment on Gradescope.
    * Please tag the pages in your PDF to the corresponding questions on Gradescope.
* Code [Notebook]:
    * Please submit the zip file generated by the final export cell in this coding notebook to the `HW 4.1 Coding [Code]` assignment on Gradescope.
    * Code submitted in your notebook must match the code in your PDF submission, and produce the *exact* same outputs if re-run by course staff. Inconsistent or incomplete code might not receive marks.
* Note: Make sure you submit *both* a PDF version of your notebook (to the Coding [PDF] assignment) and a zip file of the notebook (to the Coding [Code] assignment) on Gradescope.

### Submission Tips:
- **Plan ahead:** We strongly encourage you to submit your work several hours before the deadline. This will give you ample time to address any submission issues.
- **Reach out for help early:** If you encounter difficulties, contact course staff well before the deadline. While we are happy to assist with submission issues, we cannot guarantee responses to last-minute requests.

---

## Assignment Overview

This notebook contains a series of tasks designed to help you practice and apply key concepts in data manipulation and visualization. You will complete all the TODOs in the notebook, which include both coding and written response questions. Some tasks are open-ended, which allows you to explore and experiment with different approaches.

### Key Learning Objectives:
1. Implement your own neural networks in `PyTorch`
2. Create your own custom Datasets, DataLoaders, and training loops
3. Gain hands-on experience implementing an architecture from a paper
4. Learn about the ResNet architecture
5. Implement a transformer block in `PyTorch`
6. Understand how the key components in a transformer fit together
---

<div align="center">

## Grading Breakdown
| Question | Points |
|----------|--------|
| 1a       | 2      |
| 1b       | 4      |
| 1c       | 2      |
| 1d       | 2      |
| 1e       | 1      |
| 2a       | 2      |
| 2b       | 3      |
| 2c       | 1      |
| 2d       | 1      |
| 3a       | 1      |
| 3b       | 2      |
| 3c       | 2      |
| 3d       | 2      |
| 3e       | 3      |
| 3f       | 3      |
| 3g       | 3      |
| 3h       | 2      |
| 3i       | 2      |
| 3j       | 2      |
| 3k       | 2      |
| 3l       | 1      |
| 3m       | 1      |
| 3n       | 1      |
| 3o       | 1      |
| **Total**| **46**|

</div>

---

## Instructions:
1. Carefully read each question and its requirements.
2. Complete all TODOs in the notebook. You may add extra lines of code if needed to implement your solution.
3. For manual questions, provide clear and concise written responses.
4. Test your code thoroughly to ensure it meets the requirements.

Good luck! Autobots, roll out! 🤖

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
import random
from datasets import load_dataset
import matplotlib.pyplot as plt
import plotly
import plotly.subplots
import plotly.graph_objects
import os
import json
import re
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.transforms as transforms

### **IMPORTANT:**
- Do not change the random seed values!!!
- Make sure to turn on the GPU accelerator in Google Colab's runtime settings before starting the notebook.

In [ ]:
def set_seed(seed):
    """
    Set the seed for all random number generators
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Set random seeds for reproducible results
SEED = 42
set_seed(SEED)

IS_GRADING_ENV = os.getenv("IS_GRADING_ENV") == "true"

if torch.accelerator.is_available():
    device = torch.accelerator.current_accelerator()
else:
    device = "cpu"

print(f"Using device: {device}") # Check that the current device is cuda or another GPU accelerator!

# Subpart 1: Convolutional Neural Networks

First, let's warm up by building a CNN (Convolutional Neural Network) in PyTorch to get familiar with the PyTorch library.

### So what is PyTorch?

PyTorch is a powerful Python library for building and training neural networks. It is built around data structures called tensors (multi-dimensional arrays!). PyTorch is awesome because it handles creating computational graphs and calculating gradients for us.

#### PyTorch Modules

PyTorch's main building block are *modules*, which you can think of as different layers in your neural net. PyTorch has a couple of pre-defined classes which inherit from the base `nn.Module` class.

Modules can be as simple as individual layers:
* [Linear fully-connected layer](https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html)

... activation functions:
* [ReLU](https://docs.pytorch.org/docs/stable/generated/torch.nn.ReLU.html)

... entire blocks of a model, which often have *other* modules inside of them!
* [LSTM/long short-term memory blocks](https://docs.pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

... or complete models!
* [Transformer - we'll be implementing our own version later in this homework](https://docs.pytorch.org/docs/stable/generated/torch.nn.Transformer.html)

In short, you can think of modules as LEGO bricks that you can piece together in different ways.

#### Implementing custom `nn.Module` subclasses!

We can also define our own classes that inherit from the `nn.Module` base class! To implement your own subclass of `nn.Module`, you need to define 2 methods:
* `__init__(self, ...)`: The constructor method where you initialize all the layers and parameters your module will use. Call super().__init__() first to properly initialize the parent nn.Module class. Define any layers (e.g., Conv2d, Linear, etc.) as instance attributes here. PyTorch will automatically track these as submodules and register their parameters.

* `forward(self, x)`: Defines the forward pass computation. Takes input tensor(s) and returns output tensor(s). This is where you specify how data flows through your layers—apply operations, activations, skip connections, etc. You call your model with output = model(input), which internally invokes the forward method. PyTorch's autograd automatically builds the backward pass from your forward operations.

#### PyTorch Functionals

Another important part of PyTorch is `torch.nn.functional`. This is a module (note: "module" in this case refers to a sublibrary of a Python library. It's not the same thing as the `Module` classes we were just talking about! I know, it's confusing!) of stateless functions, which means that these functions perform operations on tensors without maintaining any state or tracking parameters. So these don't contain learnable parameters we can train and update. `torch.nn.functional` is often imported as `F`; in practice you'll often see it being called like this: `F.relu(x)`.

Often times, certain kinds of layers have both `nn.Module` and `nn.functional` implementations in PyTorch. How do you know when to use which kind? As a rule of thumb, whenever you need to the layer's parameters to be learnable (i.e. changeable over the course of training), use the `nn.Module` version. On the other hand, for operations like activation functions (e.g. ReLU or TanH) we don't have any parameters that we can learn! So their `nn.Module` and `nn.functional` version can be used interchangeably. In your code below, you can choose to use either `nn.ReLU` or `F.relu`.

| Aspect   | nn.Module | nn.Functional |
|----------|-----------------|---------------|
| State | Stateful - maintains parameters | Stateless - just functions|
| Parameters | Automatically registers learnable parameters and tracks them for optimization | No parameter tracking |
| Implementation details | Need to initialize the module first (e.g. using `self.relu = F.relu()`) before using it (e.g. `a = self.relu(x)`) | Can call it directly on an input (e.g. `a = F.relu(x)`) |
| Usage examples | `nn.Linear(10, 5), nn.ReLU()` | `F.linear(x), F.relu(x)` |

## Question 1a

Using some of PyTorch's existing `nn.Module` classes, we can build our own neural networks. Let's try building a CNN with the following architecture:

> * **Convolution Layer 1**: 16 output channels, kernel size 3 x 3, stride 2, padding 1
> * **ReLU Activation**: Element-wise ReLU
> * **Convolution Layer 2**: 16 output channels, kernel size 7 x 7, stride 2, padding 1
> * **ReLU Activation**: Element-wise ReLU
> * **Linear Classifier**: Linear layer for classification

**Task:**
1. Implement the `__init__` method.
    1. Don't forget to call `super().__init__()`!
    2. Instantiate all the layers we'll need for our CNN.
2. Implement the `forward` method.
    1. Pass the input `x` through each of the layers.
    2. Return the final output.

**Hints:**
* The documentation for the `Conv2d` class [here](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) will contain helpful information about what parameters you need to specify to instantiate a `Conv2d` layer
* Since our images have 3 channels for Red, Green, and Blue, how many input channels should our first `Conv2d` layer have?
* How many channels does the output of the 1st ReLU layer have? What should be the number of input channels for the 2nd `Conv2d` layer?
* The final linear layer expects a flattened tensor for each item in the batch. What shape is the output after the 2nd ReLU? (Hint: you can either calculate the shape explicitly using formulas for what the dimensions of a conv layer's feature map will be, or add a print statement!) How can you flatten it into a 1D tensor with shape `(Batch_size, 1)`?
    * Helpful website to calculate the output shapes after a convolution: https://madebyollin.github.io/convnet-calculator/
* The linear classifier will make predictions about which class the image belongs to by outputing a vector of dimension `(num_classes)` containing the probabilities that the image belongs to a certain class for each image in the batch. What should be the output dimension for the linear classifier?
* You can choose to either instantiate a `nn.ReLU` layer in `__init__`, or directly call the functional `F.ReLU` in your forward pass!

**Helpful Documentation**
* [`nn.Conv2d`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
* [`nn.Linear`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html)
* [`nn.ReLU`](https://docs.pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
* [`F.ReLU`](https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.relu.html)

<!-- BEGIN QUESTION -->



In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # TODO: Instantiate the layers your CNN will have!
        self.conv1 = ...
        self.conv2 = ...
        self.linear = ...
        # Optional: You can also define your ReLU layers here if you'd like
        # Otherwise, if you decide to use F.relu you can just remove these lines
        # self.relu1 = ...
        # self.relu2 = ...

    def forward(self, x):
        # TODO: 1. Pass the input through the first conv layer
        conv1_out = ...

        # TODO: 2. Apply ReLU
        relu1_out = ...

        # TODO: 3. Pass to the 2nd conv layer
        conv2_out = ...

        # TODO: 4. Apply ReLU
        relu2_out = ...

        # TODO: 5. Flatten before linear layer
        flat = ...

        # TODO: 6. Pass to linear layer
        out = ...

        return out

<!-- END QUESTION -->

### Check: CNN output shape

A helpful debugging tip while building your own deep-learning models or custom modules is to check the shapes of your outputs!

Below, we create a tensor of shape (1, 224, 224, 3) filled with random numbers between [0, 1] and pass it through our `CNN` to check if the output has the shape we expect.

How do we determine what the shape of our output should be? We can trace through what the shapes should be after each of our layers.

Assume we instantiate a `CNN` with `num_classes=10` and we pass an input of shape `(1, 3, 224, 224)` through the `CNN`.

These would be the shapes after each layer:
1. Input: `1 x 3 x 224 x 224` --> dimensions: Batch size x 3 RGB channels x height of 224 pixels x width of 224 pixels
2. After conv1: `1 x 16 x 112 x 112` --> The first convolution produces feature maps with shape 112 x 112 and increases the number of channels to 16
4. After the first ReLU: `1 x 16 x 112 x 112` --> ReLU doesn't change the output dimensions
5. After conv2: `1 x 16 x 54 x 54` --> The second convolution produces feature maps with shape 54 x 54. This convolution doesn't change the number of channels
6. After the 2nd ReLU: `1 x 16 x 54 x 54` --> ReLU doesn't change the output dimensions
7. After the linear layer: `1 x 10` --> A tensor containing the logits for each class

You should verify that the shape of your actual output is also 1 x 10!

In [ ]:
x = torch.rand((1, 3, 224, 224))  # 1 image, 3 RGB channels, 224 x 224 pixels
print(f"Shape of input: {x.shape}")

res = CNN(num_classes=10)
out = res(x)
print(f"Shape of output: {out.shape}")
assert out.shape == torch.Size([1, 10]), f"Expected output shape (1, 10), but got {out.shape}"

## Load the Data

Before we jump into training our models, let's discuss how to work with data in our training loop. First, we'll load in a image classification dataset from HuggingFace.

For this assignment, we'll use a "mini" version of ImageNet-1k from HuggingFace, which contains 100 of the original 1000 ImageNet-1k classes. The Mini Imagenet dataset comes from timm (PyTorch Image Models), which is a popular Python library that provides a comprehensive collection of pre-trained computer vision models, particularly focused on image classification.

The mini-imagenet dataset on HuggingFace has:
* 50,000 training samples
* 10,000 validation images
* 5,000 test samples (50 samples per class)

The `split` parameter in HuggingFace's `load_dataset` function specifies which portion of the dataset to load. You can load a complete split like `split='train'`, `split='validation'`, or `split='test'`. Additionally, you can load just a subset of a split using slice notation - for example, `split='train[:10%]'` will load only the first 10% of the training data, which is useful for faster experimentation and development.

We want to work with just 10 classes for learning efficiency. To simplify our experiments and ensure balanced class representation, we'll use a helper function `balanced_split` to sample exactly 1000 training images, 200 validation images, and 200 test images, with images distributed evenly between the selected classes (as much as possible). This means each class will contribute an equal number of samples, unless a class has too few images, in which case we'll include all available samples from that class.

**Notes:**
* We're only using 1000 training images, 200 validation images, and 200 test images for learning purposes. In a real-world scenario, you'd likely use a far larger dataset for training and evaluation :)
* Loading the dataset and creating a balanced split may take a couple minutes to complete.

### Understanding HuggingFace Dataset Objects

HuggingFace datasets are structured like lists of dictionaries, where each element represents one data sample. For image datasets like mini-imagenet, each sample has two key fields:

* **`image`**: A PIL Image object. This is the actual image data that can be displayed, transformed, or converted to a numpy array using `np.array(image)`. The images are typically stored in RGB format with pixel values ranging from 0-255.

* **`label`**: An integer representing the class ID for that image. For example, `label=0` might correspond to "house_finch", `label=1` to "robin", etc. The mapping between label integers and class names is typically stored in the dataset's features or metadata.

**Example usage:**
```python
# Access a single sample
sample = mini_imagenet_train[0]
print(sample.keys())  # Output: dict_keys(['image', 'label'])

# Get the image and label
image = sample['image']  # PIL Image object
label = sample['label']  # Integer class ID

# Convert image to numpy array (height, width, channels)
image_array = np.array(image)
print(image_array.shape)  # e.g., (84, 84, 3) for RGB
```

In [ ]:
# Helper function to sample a balanced subset of the dataset
def balanced_split(dataset, n_total: int, classes_to_keep: set[int] = None):
  """
  Returns a balanced subset of the dataset with `n_total` samples, equally divided among the specified classes.

  If `classes_to_keep` is provided, only samples from these classes are included, each contributing exactly (n_total // num_classes) samples.
  If not provided, all unique classes in the dataset are used.
  Assumes the dataset is a Hugging Face Dataset object with a 'label' field.

  Args:
      dataset (datasets.Dataset):
          The dataset to subsample. Must have a 'label' column.
      n_total (int):
          Total number of samples to return. Must be divisible by the number of selected classes.
      classes_to_keep (set[int], optional):
          Class labels to include in the balanced subset. Each class will have n_total // num_classes samples.
          If None, all classes in the dataset will be used.

  Returns:
      datasets.Dataset:
          A new dataset with `n_total` samples, equally distributed across the specified classes.

  Raises:
      ValueError:
          If the number of samples per class is greater than the available samples for any class,
          or if n_total is not divisible by the number of classes.
  """
  if not classes_to_keep:
    classes_to_keep = set(dataset['label'])
  n_per_class = n_total // len(classes_to_keep)

  # Dictionary of class to list of indices that correspond to that class
  cls_to_indices = {cls: [] for cls in classes_to_keep }

  # Iterate through all indices and labels in the dataset's label list
  for i, cls in enumerate(dataset['label']):
    # If this label corresponds to a class we want to keep, append it to the appropriate class's list
    if cls in cls_to_indices:
      cls_to_indices[cls].append(i)

  sampled_idxs = []
  for cls, idx_list in cls_to_indices.items():
    if len(idx_list) < n_per_class:
      # If not enough samples, include them all
      sampled_idxs.extend(idx_list)
    else:
      sampled_idxs.extend(random.sample(idx_list, k=n_per_class))

  return dataset.select(sampled_idxs)

# Load a dictionary that maps numeric class IDs to string class names
with open('imagenet_class_id_to_name.json', 'r') as file:
    class_id_to_name = json.load(file)
    class_id_to_name = {int(key): value for key, value in class_id_to_name.items()}
    print(f"Classes in the image net dataset: {class_id_to_name}")

# Load the training, validataion, and test sets
mini_imagenet_train = load_dataset("timm/mini-imagenet", split="train[:10%]")
mini_imagenet_train = balanced_split(mini_imagenet_train, n_total=1000)
print(mini_imagenet_train)

mini_imagenet_val = load_dataset("timm/mini-imagenet", split="validation[:10%]")
mini_imagenet_val = balanced_split(mini_imagenet_val, n_total=200)
print(mini_imagenet_val)

## Question 1b: Prepare the Dataset

### PyTorch Dataset and DataLoaders

Now that we have our data, we need to prepare it for model training. When we train machine learning models, we need an efficient way to access our data (often in batches, which are subsets of our data that we pass into our model all at once to take advantage of our computer/server's full computation power). PyTorch's `Dataset` and `DataLoader` classes allow us to neatly encapsulate code for processing and accessing our data.

* PyTorch's `Dataset` class is an abstract class that represents a dataset–it stores the samples and their corresponding labels.
* Once we have a `Dataset`, we can pass it into a `DataLoader`, which wraps an iterable around the `Dataset` so we can easily access samples individually or in batches in our training loop.
* PyTorch has a great tutorial about `Datasets` and `DataLoaders` you can check out [here](https://docs.pytorch.org/tutorials/beginner/basics/data_tutorial.html)

Why is PyTorch's `Dataset` class useful?
1. `Datasets` can integrate with PyTorch's `DataLoader` objects for batching, shuffling, and parallel loading.
2. You have more flexible control over how data is loaded and preprocessed.
3. You can encapsulate all data logic in one place, making your training code cleaner.
4. The `Dataset` and `Dataloader`'s standard interface means your dataset works with any PyTorch training loop.

PyTorch provides many built-in datasets (MNIST, CIFAR-10, ImageNet, etc.) through `torchvision.datasets`, but you'll need a custom Dataset when:
* Your data has a unique structure, such as being stored in a local folder on your computer.
* The data has non-standard formats (e.g. medical imaging, satellite data, audio spectrograms).
* You have special loading requirements, like combining data from multiple sources.
* You need custom preprocessing logic.

In our case, since we're loading Mini ImageNet from HuggingFace and subsampling only 10 classes out of it, let's get some hands-on practice making our own custom dataset!

To create your own custom dataset, you need to inherit from `torch.utils.data.Dataset` and implement three key methods.
1. `__init__(self, ...)`
    1. This is your custom dataset's constructor! Here, you should save your data, initialize any transforms/preprocessing, and set up any configurations you may need.
2. `__len__(self)`
    1. Returns the total number of samples in your dataset. This is used by DataLoader to know when to stop iterating.
3. `__getitem__(self, idx)`
    1. The most important method! Returns a single sample from your dataset at the given index idx. This is where you:
        1. Load the actual data (e.g., index from an array, read image from disk).
        2. Apply transformations.
        3. Return the sample and its label.

Example:
```python
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, custom_data, labels, preprocess=None):
        """
        Args:
            custom_data (str): list of data samples
            labels (list): List of labels corresponding to images
            preprocess (callable, optional): Optional preprocessing function to apply
        """
        self.data = custom_data
        self.labels = labels
        self.preprocess = preprocess
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the data at this index
        item = self.data[idx]
        # Get the label for this data point
        label = self.labels[idx]
        
        # Apply preprocessing
        if self.preprocess:
            item = self.preprocess(item)
        
        return item, label
```

**Task:** Implement the `__init__`, `__len__`, `__getitem__`, and `show_images` methods of the `MiniImageNetDataset`
* `__init__`:
    * Save the input arguments (`dataset`, `class_id_to_name`, `split`, and `transform`) as instance attributes (`self.dataset`, etc.)
* `__len__`:
    * Return the length of the dataset
* `__getitem__`:
    * Get the item in the HuggingFace dataset at the given index.
    * Turn the item's image into a 3D numeric array. Cast the image into a `torch.Tensor` with `dtype=torch.float`, permute the image into the shape (C, H, W), and apply the transform if it's not `None`.
    * Cast the label to a `torch.Tensor` with `dtype=torch.long`.
    * Return the transformed image and label.
* `show_images`:
    * Sample `num_images` random images from the dataset.
    * Display the images in a grid layout.
    * For each image, use `class_id_to_name` to get a readable class name and set the readable class name as the title of the image.
    * You can use matplotlib or plotly to display the images.
    
**Hints:**
* It might be helpful to refer to the section on "Understanding HuggingFace Dataset Objects" to understand how to extract an image and its label from the dataset.
* In the `__getitem__` method, we should make sure the item and label we return are both of type `torch.Tensor`. If the item and label aren't `torch.Tensors`, you should:
    * Cast the item to `torch.Tensor` with `dtype=torch.float`
    * Cast the label to `torch.Tensor` with `dtype=torch.long`
    * Note that PyTorch expects the inputs to the models to have `dtype=torch.float` to match the dtype of the model weights, and the labels to have `dtype=torch.long` when we pass them into loss functions.
* Make sure that you permute the image into the shape (C, H, W) where C = number of channels (usually 3 for R, G, and B), H = image height, and W = image width.
    * Images are usually represented as arrays in the shape (H, W, C) but libraries like PyTorch usually expect each image to have shape (C, H, W) to optimize the matrix math that goes on behind the scenes! For instance, when we visualize nd.arrays using Plotly or Matplotlib, your image arrays should have the shape (H, W, C). But when we want to use arrays/tensors to train machine learning models, your image arrays should have the shape (C, H, W).
    * [`torch.permute`](https://docs.pytorch.org/docs/stable/generated/torch.permute.html#torch.permute) might be helpful.

<!-- BEGIN QUESTION -->



In [ ]:
class MiniImageNetDataset(Dataset):
    def __init__(
        self,
        dataset,
        class_id_to_name: dict[int, str],
        split: str = "train",
        transform: transforms.Compose = None,
    ):
        """
        Args:
            dataset (datasets.Dataset): HuggingFace dataset object
                Each sample in the dataset has:
                - An "image" field that contains the image data
                - A "label" field that contains the numeric class ID
            class_id_to_name (dict): dictionary mapping numeric class IDs to class names
            split (str): "train" or "val" or "test"
        """
        # TODO: Implement the __init__ method of the MiniImageNetDataset
        self.split = ...
        self.dataset = ...
        self.transform = ...
        self.class_id_to_name = ...

    def __len__(self):
        # TODO: Implement the __len__ method of the MiniImageNetDataset
        ...

    def __getitem__(self, index):
        """
        Returns a single sample from the dataset at the given index.
        Args:
            index (int): index of the item to get
        Returns:
            tuple: (image, label) where image is a tensor of shape C x H x W and label is a tensor of shape 1
        """

        # TODO: Implement the __getitem__ method of the MiniImageNetDataset
        # Don't forget to...
        # - Cast the item and label to torch.Tensor with the correct dtype
        # - Permute the image into the shape (C, H, W)
        # - Apply the transform if it's not None
        ...
        return item, label

    def show_images(self, num_images: int = 10):
        """
        Visualize images in a grid layout.

        Args:
            num_images (int): Number of images to display per class

        Returns:
            tuple: (fig, ax) - Matplotlib figure and axes objects
        """
        # TODO: Implement the show_images method
        # Make sure to call fig.show() or plt.show() to display your grid of images at the end!
        ...

<!-- END QUESTION -->

## Question 1c: Create Datasets and DataLoaders

Using the custom Dataset class we implemented, let's create MiniImageNetDatasets that contain our training and validataion data.

In our Dataset implementation, we specified a `transform` parameter that we can use to apply transformations to our images. For most vision-tasks, we often apply transformations to our images to help the model generalize better.

PyTorch has a `transforms` module that contains many built-in transformations you can use. Additionally, we can compose multiple transformations into a single `transforms.Compose` object.

For this assignment, we'll use the following transformations:
* `transforms.Resize`: Resize the image to a smaller size
* `transforms.RandomCrop`: Randomly crop the image to a smaller size
* `transforms.RandomHorizontalFlip`: Randomly flip the image horizontally
* `transforms.ToTensor`: Convert the image to a tensor

In the following block, we've defined the standard preprocessing pipeline for ImageNet models:
* Resize images to 256 pixels on the shorter side.
* Crop to 224×224 (the standard input size for most ImageNet-trained architectures like ResNet).
* For training, we use data augmentation with random crops and horizontal flips to help the model generalize better.
* For validation we use deterministic center crops to ensure consistent evaluation.
* Lastly, we normalize the images using means and standard deviations computed from the entire ImageNet training set. The mean `[0.485, 0.456, 0.406]` and standard deviations `[0.229, 0.224, 0.225]` have 3 values, one for each channel (R, G, B). By normalizing our Mini-ImageNet data using these same statistics, we ensure that the pixel value distributions match what models expect if we were to use pretrained ImageNet weights for transfer learning. This normalization also helps with training stability by centering the data around zero.

**Task:** Create training and validation instances of our custom dataset class using the appropriate transformations.
* Define a `train_dataset` that uses `mini_imagenet_train` as the dataset, `class_id_to_name` as the class_id_to_name, `split="train"`, and `transform=train_transform`.
* Define a `val_dataset` that uses `mini_imagenet_val` as the dataset, `class_id_to_name` as the class_id_to_name, `split="val"`, and `transform=val_transform`.

<!-- BEGIN QUESTION -->



In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(256),                  # Resize the shorter side to 256
    transforms.RandomCrop(224),              # Random crop into shape 224×224
    transforms.RandomHorizontalFlip(),       # Random horizontal flip
    transforms.ToTensor(),                   # Convert to tensor, scale to [0,1], and permute into shape (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) # Normalize
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = ...
val_dataset = ...

<!-- END QUESTION -->

Then, we will create DataLoaders for our Datasets.

**Task:** Create a dataloader for our training, validation, and test datasets. Use a `batch_size` of 32, and set `shuffle=True`.

<!-- BEGIN QUESTION -->



In [ ]:
train_dataloader = ...
val_dataloader = ...
...

# Print the first batch of data and labels
for batch in train_dataloader: # each batch is a tuple of (data, labels)
    data = batch[0] # data is a tensor of shape (B, 3, 224, 224)
    labels = batch[1] # labels is a tensor of shape (B,)
    print(f"Shape of data: {data.shape}")
    print(f"Shape of labels: {labels.shape}")
    break

<!-- END QUESTION -->

## Question 1d: Train your CNN

We are ready to train our CNN model! To train our CNN model, we need to write a training loop that passes inputs into our model, updates the weights of our model, and repeats!

**Important terminology**:
* *epoch:* one complete pass through the entire training dataset
* *batch:* a subset of samples we pass into the model at the same time
* *optimizer:* what kind of optimization algorithm we want to use (e.g. Adam, Momentum)
* *criterion:* the loss function we want to use to evaluate our model on

### Training Loops
A training loop is where your model actually learns! Each iteration through the loop, the model:
* Makes predictions
* Calculates how wrong it was (loss)
* Updates its weights to do better next time

### PyTorch Training Loops: Step-by-Step Guide and Best Practices
1. Setup - Move model to device
    1. It's a good practice to begin by moving your model to the device (either `cuda` aka GPU or `cpu`). Your model and data must be on the same device. Moving the model to the GPU can speed up training.
    
> ```python
> model.to(device)  # device = 'cuda' or 'cpu'
> ```

2. Outer Loop - Iterate for `num_epochs` epochs
    1. Models rarely learn perfectly in one pass. Multiple epochs let the model see the data repeatedly and gradually improve.

>```python
>for epoch in range(num_epochs):
>    # An epoch = one complete pass through the entire training dataset
>```

3. Set the model to training mode before passing in training data
    1. Some layers (Dropout, BatchNorm) behave differently during training vs testing.

>```python
>model.train()
>```

4. Inner loop - Iterate over training batches
    1. Process data in batches to make the most of your GPU's computational power without overflowing memory.
    2. Move the data to the same device as the model, and if needed, cast them to the appropriate dtypes.
        1. Data should have the same dtype as your model's parameters.
        2. Usually, loss functions expect the target labels to have `dtype=torch.long`.

>```python
>for x, y in train_dataloader:
>    # Process one batch at a time
>    x = x.to(device, dtype=torch.float)
>    y = y.to(device, dtype=torch.long)
>```

5. Zero out the gradients - PyTorch accumulates gradients by default. If you don't reset them, old gradients will add to new ones, causing wrong updates! **Always** zero out the gradients BEFORE the forward pass of each batch.

>```python
>optimizer.zero_grad()
>```

6. Make a forward pass to get your model's predictions on the input data.

>```python
>y_hat = model(x)
>```

7. Compute the loss between your predictions and the target labels
    1. Loss measures how wrong your predictions are. Lower loss = better predictions.
    2. The criterion will return a single number representing the loss.

>```python
>loss = criterion(y_hat, y)
>```

8. Backward pass to compute gradients
    1. Call `loss.backward()` to compute gradients for each learnable weight in your model.
9. Update the weights
    1. Call `optimizer.step()` to update the weights in your model based on the gradients computed during backpropagation!
10. Track training metrics (optional but a good practice!)
    1. Track the training loss and training accuracy
    2. Since loss is a `torch.Tensor`, we can get its scalar value by calling `loss.item()`. Otherwise, we can't directly add a `torch.Tensor` (even if it's just a one-dimensional scalar!) to another Python primitive like `train_loss`, which might be a Python float.
    3. In multi-class classification, the model's output `y_hat` will be an array of length `num_classes` where each element is the probability that the given data point belongs to that class. To get the class with the highest probablity, we can call `argmax` (returns the index corresponding the maximum value). Remember that we are using batches in our training loop! So `y_hat` will be a tensor of shape (batch_size, num_classes). By specifying `dim=1`, we can get the predicted class for each item in the batch. So `preds` will have shape (batch_size, 1) --> one predicted class for each item in the batch.
    4. To compute the training accuracy, we can check when the predicted class equals the actual true targets `y` and sum up all the correct predictions. Since `preds` and `y` are both tensors, the expression `(preds == y)` and the sum is also a `torch.Tensor`. So similarly to the loss, we can get the scalar value of this tensor by calling `.item()`.

>```python
>train_loss += loss.item()  # Accumulate loss
>preds = torch.argmax(y_hat, dim=1)  # Get predicted class for each item in the batch
>train_correct += (preds == y).sum().item()  # Count correct predictions
>```

11. Measure validation metrics after each epoch
    1. Set the model to `eval` mode - Disables dropout, uses running stats for BatchNorm, etc.
    2. Make predictions on the validation data, and measure validation loss and/or accuracy.
    3. **Important note:** Make sure to do all your validation under a `with torch.no_grad()` header! This tells PyTorch not to keep track of forward pass operations or compute gradients during this code block. This saves memory and speeds up validation because we don't want to train the model during validation–we want to measure how it's performing.

Complete training loop structure (simplified)

>```python
># === SETUP ===
>model.to(device)
>
># === EPOCH LOOP ===
>for epoch in range(num_epochs):
>    train_loss = 0.0
>    train_correct = 0.0
>    
>    # === TRAINING PHASE ===
>    model.train()  # Enable training mode
>    
>    for x, y in train_dataloader:
>        # 1. Move data to device
>        x, y = x.to(device), y.to(device)
>        
>        # 2. Reset gradients
>        optimizer.zero_grad()
>        
>        # 3. Forward pass
>        y_hat = model(x)
>        
>        # 4. Compute loss
>        loss = criterion(y_hat, y)
>        
>        # 5. Backward pass (compute gradients)
>        loss.backward()
>        
>        # 6. Update weights
>        optimizer.step()
>        
>        # 7. Track metrics (optional)
>        # ...
>    
>    # === VALIDATION PHASE ===
>    model.eval()  # Disable training-specific layers
>    with torch.no_grad():  # Don't compute gradients
>        for x, y in val_dataloader:
>            # Forward pass only, no backward pass!
>            y_hat = model(x)
>            loss = criterion(y_hat, y)
>            # Track validation metrics
>```


### How we calculate average loss and accuracy
* When we train a model, we want to report the average loss and average accuracy for each epoch, not just the total. This helps us understand how well the model is performing per sample, and lets us compare results across different epochs and experiments.

**Loss Calculation**
* During each epoch, for each batch we encounter, we add the loss computed over all items in the batch to the total loss: `train_loss += loss.item()`.
* At the end of the epoch, we divide by the number of batches: `train_loss /= len(train_dataloader)`.
* Each batch's loss is an average over the samples in that batch, so dividing by the number of batches gives us the average loss per batch for the whole epoch.

**Accuracy Calculation**
* We count the total number of correct predictions for all samples in the epoch: `train_correct += (preds == targets).sum().item()`.
* At the end of the epoch, we divide by the total number of samples in the dataset: `train_acc = train_correct / len(train_dataloader.dataset)`.
* This gives us the fraction of samples the model predicted correctly (i.e., the overall accuracy for the epoch).

In the following block, we've implemented a training loop for you!

**Note:** `tqdm` is a Python library used to display progress bars. In the `train` function we've implemented, we use it as a wrapper around iterators such as the dataloaders so that we can track the progress of how many epochs we've trained for. It's a nice-to-have, but when you implement your own training loops in the future, you can also simply iterate through the dataloaders like in the simplified example above too.

In [ ]:
# Common practice to check if GPU is available
if torch.accelerator.is_available():
    device = torch.accelerator.current_accelerator()
else:
    device = "cpu"
print(f"Using device: {device}")
assert str(device) in ["cuda", "mps"], "Please make sure you are using an accelerator (e.g. GPU or MPS)"


def train(
    model, optimizer, criterion, num_epochs, train_dataloader, val_dataloader=None
):
    """
    Args:
        model: the model to train
        optimizer: the optimizer to use
        criterion: the loss function to use
        num_epochs: the number of epochs to train for
        train_dataloader: the dataloader for the training set
        val_dataloader: the dataloader for the validation set
    """
    # === SETUP ===
    model.to(device)

    # == (Optional) Perform a validation loop before training as a baseline ==
    if val_dataloader:
        model.eval()  # Set model to evaluation mode
        num_correct = 0.0
        val_loss = 0.0
        with torch.no_grad():  # Don't compute gradients
            for x, y in val_dataloader:
                x = x.to(device, dtype=torch.float)
                y = y.to(device, dtype=torch.long)
                y_hat = model(x)

                loss = criterion(y_hat, y)  # pred, target
                val_loss += loss.item()

                preds = torch.argmax(y_hat, dim=1)
                num_correct += (preds == y).sum().item()

        val_acc = num_correct / len(val_dataloader.dataset)
        tqdm.write(f"Initial validation loss: {val_loss:.4f}")
        tqdm.write(f"Initial validation accuracy: {val_acc:.4f}")

    # Lists to store metrics across epochs
    train_accuracies = []
    train_losses = []
    val_accuracies = []
    val_losses = []

    epoch_pbar = tqdm(range(num_epochs), desc="Training Progress", position=0)

    # === EPOCH LOOP ===
    for epoch in epoch_pbar:
        # === TRAINING PHASE ===
        model.train()  # Set model to training mode

        # Initialize metrics for this epoch
        train_loss = 0.0
        train_correct = 0.0

        # === INNER LOOP (iterate over training batches) ===
        for x, y in train_dataloader:
            # 1. Move data to device
            x = x.to(device, dtype=torch.float)
            y = y.to(device, dtype=torch.long)

            # 2. Reset gradients
            optimizer.zero_grad()

            # 3. Forward pass
            y_hat = model(x)

            # 4. Compute loss
            loss = criterion(y_hat, y)

            # 5. Backward pass (compute gradients)
            loss.backward()

            # 6. Update weights
            optimizer.step()

            # 7. Track metrics
            train_loss += loss.item()  # Accumulate loss (convert to Python float)
            preds = torch.argmax(
                y_hat, dim=1
            )  # Get predicted class for each item in the batch (highest prob = highest confidence)
            train_correct += (preds == y).sum().item()  # Count correct predictions

        # === END OF INNER LOOP ===

        # Compute average metrics for the epoch
        avg_train_loss = train_loss / len(
            train_dataloader
        )  # Average loss = total loss / total number of batches
        train_acc = (
            train_correct / len(train_dataloader.dataset)
        )  # Average accuracy = accuracy / total number of data points seen across the whole epoch (i.e. in the entire dataset)
        train_losses.append(avg_train_loss)
        train_accuracies.append(train_acc)

        print(f"\nEpoch {epoch + 1} - Training accuracy: {train_acc:.3f}")
        print(f"Epoch {epoch + 1} - Training loss: {train_loss:.3f}")

        # === END OF TRAINING PHASE ===

        # === VALIDATION PHASE ===
        if val_dataloader:
            model.eval()  # Set model to evaluation mode
            val_loss = 0.0
            val_correct = 0.0

            with torch.no_grad():  # Don't compute gradients
                # === INNER LOOP (iterate over validation batches) ===
                for x, y in val_dataloader:
                    # 1. Move data to device
                    x = x.to(device, dtype=torch.float)
                    y = y.to(device, dtype=torch.long)

                    # 2. Forward pass only, no backward pass!
                    y_hat = model(x)

                    # 3. Compute loss
                    loss = criterion(y_hat, y)

                    # 4. Track validation metrics
                    val_loss += loss.item()
                    preds = torch.argmax(y_hat, dim=1)
                    val_correct += (preds == y).sum().item()

                # === END OF INNER LOOP ===

                # Compute average metrics for the epoch
                avg_val_loss = val_loss / len(val_dataloader)
                val_acc = val_correct / len(val_dataloader.dataset)

                val_losses.append(avg_val_loss)
                val_accuracies.append(val_acc)

            # === END OF VALIDATION PHASE ===

        # === END OF EPOCH LOOP ===

        # Print metrics for the epoch
        print(f"Epoch {epoch + 1} - Validation accuracy: {val_acc:.3f}")
        print(f"Epoch {epoch + 1} - Validation loss: {val_loss:.3f}")

        epoch_pbar.set_postfix(
            {
                "train_loss": f"{train_loss:.3f}",
                "train_acc": f"{train_acc:.3f}",
                "val_loss": f"{val_loss:.3f}",
                "val_acc": f"{val_acc:.3f}",
            }
        )

    return train_accuracies, train_losses, val_accuracies, val_losses

**Task:**
1. Instantiate a CNN model which will classify which of the 10 classes from our Mini Imagenet dataset an image belongs to.
2. Use `AdamW` as the optimizer.
    1. [PyTorch documentation for AdamW](https://docs.pytorch.org/docs/stable/generated/torch.optim.AdamW.html)
3. Use `CrossEntropyLoss` as our criterion.
    1. [PyTorch documentation for CrossEntropyLoss](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
4. Train your CNN model for **20** epochs! You should be able to reach ~90% training accuracy after 20 epochs.

**Hints:**
* When you instantiate an optimizer, you also need to pass in what parameters you want this optimizer to update. You can access the parameters of any class that inherits from `nn.Module` by calling its `.parameters()` function!
* Make sure that you turn on a GPU runtime before training. It may take a few minutes to complete all 20 epochs of training. For reference, it took me (Deena) about 15 seconds to train my model for 20 epochs on Google Colab's T4 GPU :)

**Notes:**
* You may notice that the validation set's training accuracy lags below the training accuracy or jumps around a lot. Since we're using a toy set with only 1,000 training images and 200 validation images, that is to be expected! These metrics are an indicator that our model might be overfitting to our training set.

<!-- BEGIN QUESTION -->



In [ ]:
# TODO: Instantiate a CNN model
cnn = ...

# TODO: Set the number of epochs
num_epochs = ...

# TODO: Set the optimizer
optimizer = ...

# TODO: Set the loss criterion
criterion = ...

# TODO: Train the model

cnn_train_accuracies, cnn_train_losses, cnn_val_accuracies, cnn_val_losses = ...

<!-- END QUESTION -->

## Question 1e: Visualize the CNN's Training Performance

Let's make some plots to see how our model's loss and accuracy (hopefully) improved over the course of its training!

**Task:** Implement the `plot_loss_and_accuracy` function below to plot the CNN's training loss, training accuracy (if provided), validation loss, and validation accuracy (if provided).
* The loss/accuracy should be on the y-axis, and the number of epochs should be on the x-axis.
* You can use Matplotlib or Plotly.
* When only the training loss and validation loss are provided, your function should produce 2 total plots in total: (1) training loss, and (2) validation loss.
* When all 4 metrics are provided, your function should produce 4 total plots in total: (1) training loss, (2) training accuracy, (3) validation loss, and (4) validation accuracy.

<!-- BEGIN QUESTION -->



In [ ]:
def plot_metrics(train_losses, val_losses, train_accuracies=None, val_accuracies=None, num_epochs=None, title=""):
    """
    Plots the training loss, training accuracy, validation loss, and validation accuracy.
    Args:
        train_losses: list of training losses
        val_losses: list of validation losses
        train_accuracies: list of training accuracies
        val_accuracies: list of validation accuracies
        num_epochs: number of epochs
        title: title of the plot
    """

plot_metrics(cnn_train_losses, cnn_val_losses, cnn_train_accuracies, cnn_val_accuracies, num_epochs, "CNN Training Metrics")

<!-- END QUESTION -->

# Subpart 2: The ResNet Architecture

Deep neural networks can be incredibly powerful and expressive, but they can also be difficult to train! There are a few problems that deep neural networks encounter:
* Vanishing gradients: Earlier layers receive very small gradients because the upstream gradients get multiplied by many local gradients. Thus, during gradient descent, their weights move by very small amounts and they don't "learn" as effectively.
* Accuracy degradation: Adding more layers to a deep model causes accuracy to saturate, contributing to higher *training error* and *test error* compared to a shallower network with less layers.

Normalization can help address the vanishing gradient problem. But accuracy degradation can still persist in deep networks.

## Residual Learning

In 2015, He et al. proposed a solution to the degradation problem by introducing the concept of **residual learning**.

> [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385) Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun

Neural networks can be thought of as being composed of many different "blocks", which are just 1 or more layers (e.g. linear layers, activation functions, convolution layers, etc.) we think of as a single group.

Recall that we can view the blocks in our neural networks as functions mapping some input $x$ to some output $z$ (note that $z$ might not be the final output of our neural network like a class prediction, but could also be the input into another hidden layer).

Let's denote the ground truth function we want this block in our neural network to learn as $f$, where $f(x) = z$.

Instead of forcing the layer(s) in our block to learn $f(x)$, we let them learn another mapping $g(x)$ where $g(x) = f(x) - x$. Then, we can "add in" our original input $x$ to the output of $g(x)$ to achieve our desired mapping $f(x)$!

The authors showed that with **residual learning**:
1. Deep residual networks were easier to optimize than deep networks without residual connections
2. Residual networks demonstrate significant accuracy gains even as they get deeper, avoiding the degradation problem
3. They were able to win 1st place awards in the ImageNet detection, ImageNet localization,
COCO detection, and COCO segmentation in ILSVRC & COCO 2015 competitions.

## Benefits of Residual Learning

You might ask why we want to do this! Aren't we just trying to learn some other function $g(x)$ now? Why are we complicating things by subtracting $x$ from $f(x)$ if we have to add it back in anyways?

There are a few intuitions behind why the ResNet architecture is very powerful!

**Reason 1:** Adding a residual connection lets more of the upstream gradient flow backwards

> Imagine layer N, whose input is $x$ and whose output is $z$. Denote the upstream gradient that backpropagates from the loss $L$ to the current layer N as $\frac{\partial L}{\partial z}$. Without a residual connection, the downstream gradient layer N propagates down to layer N - 1 is: $\frac{\partial L}{\partial z} \cdot \frac{\partial z}{\partial x}$.

> With a residual connection, the downstream gradient that layer N propagates down to layer N - 1 is now $\frac{\partial L}{\partial z} \cdot (\frac{\partial z}{\partial x} + 1)$. Thus, even if the local gradient term $\frac{\partial z}{\partial x}$ is very small, the + 1 ensures that gradients don't vanish completely and allow information to flow backwards more effectively!


**Reason 2:** Blocks can learn residual mappings!

<div align="center">
<img src="https://i.imgur.com/Pxbe1Op.jpeg" alt="Residual Block" width="400"/>
<br>
<caption>
<a href="https://d2l.ai/chapter_convolutional-modern/resnet.html">
Source: Dive Into Deep Learning
</a>
</caption>
</div>

Let's zoom in on a block of a neural network (outlined by a the dotted box), with input $x$. We assume that $f(x)$, the desired ground truth mapping we want the block to learn, will be used as the input to the activation function at the top. The "weight layers" represented by the white rectangles can be fully connected linear layers or convolutional layers.

On the left, we have a block without a residual connection. The dotted-line box must directly learn $f(x)$. On the right, the layers in the dotted-line box have to learn the **residual mapping** $g(x) = f(x) - x$.

If the actual desired function we want to learn is the identity function ($f(x) = x$), then the residual mapping is just $g(x) = 0$ very easy for the layers on the right to learn–we just need to push the weights and biases of the "weight layer" to 0.

In conclusion, sometimes, a neural network has already learned everything it needs from its data to perform its task! Adding additional layers is like forcing the network to keep "squeezing" more information out from the data. Using a residual connection lets a network block decide whether or not it might be more effective to simply pass only $x$ forward, without applying additional non-linearities onto it.

## Question 2a: Implement a Residual Block in PyTorch

**Task:** Use PyTorch to instantiate a Residual Block with the following architecture:
> * Convolution layer 1 with 64 channels, 3x3 kernel, padding = 1, stride = 1 (set stride = 2 if downsampling)
> * Batchnorm
> * ReLU activation
> * Convolution layer 2 with 64 channels, 3x3 kernel, padding = 1, stride = 1
> * Batchnorm
> * Residual AKA skip connection
> * ReLU activation

**Notes about downsampling:**
* **Downsampling** refers to reducing the size of the feature map. For instance, let's say our original input to this layer was a 28 x 28 image. We can downsample it to a 14 x 14 image, essentially halving the H and W. Downsampling is often combined with an increase in the number of output channels. The main benefits of downsampling are 1) reducing computation, 2) increasing the receptive field of the output units (i.e. how many input pixels each output unit "looks" at), and 3) enabling hierarchical feature learning (i.e. letting the model learn fine details at high resolutions, then reducing the resolution by making the input that gets passed down to later layers of a smaller size so that the model can learn bigger-picture details at a lower resolution). Many ResNet implementations downsample the input in the first convolution layer. Downsampling can be implemented by using a stride of 2 instead of a stride of 1 in a Conv2d layer. This means the kernel will shift over by 2 pixels each time instead of 1.
* When downsampling, we can't directly add the input to the output of the convolution layers anymore because the dimensions won't match! Instead, we can also perform downsampling during the skip connection by applying a 1x1 convolution with stride=2 on the skip connection to match the shape of the convolution output before adding them together.

**Hints:**
1. Use separate batchnorm layers after each convolution layer!
2. `BatchNorm2d` normalizes each channel independently over the batch and spatial dimensions, so it needs one $\gamma$ and $\beta$ per channel. When initializing a `BatchNorm2d` layer, you should pass in the number of input channels as the `num_features` argument.
3. The residual connection and the output of the 2 convolutions should have the same shape (i.e. the same C, H, and W dimensions). What is the shape of the output after the 2 convolutions?
    1. To make sure the dimensions match, you can apply a 1x1 convolution to the input $x$. Think about how a 1x1 convolution affects the H, W of the input, and how you can modify it to match the C dimension of the convolution output.
4. If we want to downsample an image, we typically apply a stride of 2 in the first conv layer. If `initial_downsample = True`, make sure to adjust the stride in your conv layers in the `__init__` method appropriately! Also make sure to use a stride of 2 in the residual connection if we downsample.

Helpful website to calculate the output shapes after a convolution: https://madebyollin.github.io/convnet-calculator/

<!-- BEGIN QUESTION -->



In [ ]:
class ResidualBlock(nn.Module):
    def __init__(
        self,
        in_channels=3,
        out_channels=64,
        kernel_size=3,
        initial_downsample=False, # whether to downsample the input
        verbose=False, # whether to print debug statements
    ):
        super().__init__()
        self.verbose = verbose


        # TODO: Define the first conv layer
        self.conv1 = ...
        # TODO: Define the first batchnorm layer
        self.batchnorm1 = ...
        # TODO: Define the second conv layer
        self.conv2 = ...
        # TODO: Define the second batchnorm layer
        self.batchnorm2 = ...
        # TODO: Define the residual connection
        self.residual_connection = ...

    def forward(self, x):
        # TODO: Implement the forward pass
        conv1 = ...
        if self.verbose:
            print(f"conv1 shape: {conv1.shape}")
        bn1 = ...
        if self.verbose:
            print(f"bn1 shape: {bn1.shape}")
        z1 = ...

        conv2 = ...
        if self.verbose:
            print(f"conv2 shape: {conv2.shape}")
        bn2 = ...
        if self.verbose:
            print(f"bn2 shape: {bn2.shape}")

        residual = ...
        if self.verbose:
            print(f"residual shape: {residual.shape}")

        out = ...
        # Don't forget to apply ReLU activation after adding the residual
        out = ...
        return out

<!-- END QUESTION -->

### Check: Residual Block output shape

Below, we create a tensor of shape (1, 224, 224, 3) filled with random numbers between [0, 1] and pass it through our `ResidualBlock`. Let's do some math to determine what the expected dimensions of our `ResidualBlock` output should be!

Let's say we instantiate a `ResidualBlock` with 3 input channels, 64 output channels per hidden convolution, a kernel size of 3, and initial downsampling during the first convolution. These would be the shapes after each layer:
1. Input: 1 x 3 x 224 x 224 --> dimensions: Batch size x 3 RGB channels x height of 224 pixels x width of 224 pixels
2. After conv1: 1 x 64 x 112 x 112 --> the first convolution downsamples the image to half its original height and width, and also increases the number of channels to 64
3. After bn1: 1 x 64 x 112 x 112 --> batchnorm doesn't change the output dimensions
4. After the first ReLU: 1 x 64 x 112 x 112 --> ReLU doesn't change the output dimensions
5. After conv2: 1 x 64 x 112 x 112 --> the second conv has padding = 1. This is called *same padding* because we pad the input so that the output of this layer has the same dimensions.
6. After batch norm 2: 1 x 64 x 112 x 112 --> batchnorm doesn't change the output dimensions
7. After adding the residual: 1 x 64 x 112 x 112
8. After the final ReLU: 1 x 64 x 112 x 112 --> ReLU doesn't change the output dimensions

You should verify that the shape of your actual output is also 1 x 64 x 112 x 112!

In [ ]:
x = torch.rand((1, 3, 224, 224))  # 1 image, 3 RGB channels, 224 x 224 pixels
print(f"Shape of input: {x.shape}")

res = ResidualBlock(in_channels=3, out_channels=64, kernel_size=3, initial_downsample=True, verbose=False)
resblock_out = res(x)
print(f"Shape of output: {resblock_out.shape}")
assert resblock_out.shape == torch.Size([1, 64, 112, 112]), f"Expected output shape (1, 64, 112, 112), but got {resblock_out.shape}"

In [ ]:
# Verify the shapes of the weights
conv1_shape = res.conv1.weight.shape
print(f"Shape of conv1 weight: {conv1_shape}")
assert conv1_shape == torch.Size([64, 3, 3, 3]), f"Expected Conv1 weights to have shape (64, 3, 3, 3), but got {conv1_shape}"

conv2_shape = res.conv2.weight.shape
print(f"Shape of conv2 weight: {conv2_shape}")
assert conv2_shape == torch.Size([64, 64, 3, 3]), f"Expected Conv2 weights to have shape (64, 64, 3, 3), but got {conv2_shape}"

residual_shape = res.residual_connection.weight.shape
print(f"Shape of residual weight: {residual_shape}")
assert residual_shape == torch.Size([64, 3, 1, 1]), f"Expected residual connection to have shape (64, 3, 1, 1), but got {residual_shape}"

## Question 2b: ResNet-18(9)

Awesome! We've put together one building block of a ResNet. Now, let's implement a full ResNet-18 architecture using PyTorch.

ResNet-18 is an implementation of ResNet that has 18 total learnable layers: an initial 7x7 convolution, a series of Residual Blocks with 2 convolutions each, and a final fully connected layer for classification. There also exist other models in the ResNet family such as ResNet34, ResNet 50, and even ResNet 152!

### ResNet 18 Architecture

ResNet-18 has 4 main stages, each with a different number of residual blocks. You should use the `ResidualBlock` class you implemented above to build the ResNet-18 architecture.

---

#### 1. Initial Convolution Layer
- **Convolution Layer:** 3 input channels, 64 output channels, kernel size 7×7, stride 2, padding 3
- **BatchNorm:** Normalize 64 channels
- **ReLU Activation:** Apply ReLU after BatchNorm
- **MaxPooling:** 3×3 kernel, stride 2, padding 1

#### 2. Stage 1: 2 Residual Blocks (64 channels)
- **Residual Block 1:**
  - Input: 64 channels → Output: 64 channels
  - Kernel size: 3×3, stride 1
  - Include skip connection with 1×1 conv for dimension matching
- **Residual Block 2:**
  - Input: 64 channels → Output: 64 channels  
  - Kernel size: 3×3, stride 1
  - Include skip connection

#### 3. Stage 2: 2 Residual Blocks (128 channels)
- **Residual Block 3:**
  - Input: 64 channels → Output: 128 channels
  - Kernel size: 3×3, stride 2 (downsampling)
  - Include skip connection with 1×1 conv for dimension matching
- **Residual Block 4:**
  - Input: 128 channels → Output: 128 channels
  - Kernel size: 3×3, stride 1
  - Include skip connection

#### 4. Stage 3: 2 Residual Blocks (256 channels)
- **Residual Block 5:**
  - Input: 128 channels → Output: 256 channels
  - Kernel size: 3×3, stride 2 (downsampling)
  - Include skip connection with 1×1 conv for dimension matching
- **Residual Block 6:**
  - Input: 256 channels → Output: 256 channels
  - Kernel size: 3×3, stride 1
  - Include skip connection

#### 5. Stage 4: 2 Residual Blocks (512 channels)
- **Residual Block 7:**
  - Input: 256 channels → Output: 512 channels
  - Kernel size: 3×3, stride 2 (downsampling)
  - Include skip connection with 1×1 conv for dimension matching
- **Residual Block 8:**
  - Input: 512 channels → Output: 512 channels
  - Kernel size: 3×3, stride 1
  - Include skip connection

#### 6. Classification Head
- **Global Average Pooling:** 4x4 kernel, stride 1 (or AdaptiveAvgPool2d)
- **Flatten Layer:** Convert the the output of the global average pooling to a 1D value for each channel
- **Fully Connected Layer:** 512 input features → num_classes output

---

#### Key Implementation Notes
- **Downsampling:** The first residual block in stages 2, 3, and 4 use stride=2 to downsample the input while simultaneously increasing the number of channels. Combining downsampling with an increase in the channel count allows the network to extract more high-level features and reduces computational costs because smaller feature maps are faster to process.
- **Channel Changes:** The first residual block in stages 2-4 double the channel count of the input
- **Skip Connections**: Use 1×1 convolutions when the number of input/output channels differ

---

**Task:** Fill in the `__init__` and `forward` method of the ResNet18 class.

**Hints:**
1. `nn.Sequential` is a very helpful PyTorch class when we need to group together many layers that we want to apply one after another. Try using `nn.Sequential` to group together different blocks in each stage! You can find the docs and examples for `nn.Sequential` [here](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html).

2. Note that the stages are composed of multiple residual blocks in sequence. In the ResNet-18 implementation, whenever we want to downsample or change the number of channels, we do that in the first residual block of each stage.

3. You can use AdaptiveAvgPool2d to get a 1x1 feature map for the final fully-connected layer without having to specify the input_size, kernel_size, stride, padding, etc. explicitly.

4. You can use nn.Flatten() to flatten the output of the global average pooling layer to a 1D value for each channel.

<!-- BEGIN QUESTION -->



In [ ]:
class ResNet18(nn.Module):
    def __init__(self, num_classes, verbose=False):

        # TODO: Instantiate all the layers/stages of ResNet-18
        # Hint: don't forget to call super().__init__() first!
        ...

    def forward(self, x):

        # TODO: Implement the forward pass of ResNet-18
        ...

        return out

<!-- END QUESTION -->

### Check: ResNet output shape

Once again, let's check the output shape of our ResNet.

What should be the output's expected shape?

If the dimensions and channels of all our intermediate convolution layers match up, we should get an output with shape (Batch_size, 10) since we specified 10 as the number of classes!

In [ ]:
x = torch.rand((1, 3, 224, 224))  # 1 image, 3 RGB channels, 224 x 224 pixels
print(f"Shape of input: {x.shape}")

resnet18 = ResNet18(num_classes=10, verbose=True)
resnet18_out = resnet18(x)
print(f"Shape of output: {resnet18_out.shape}")
assert resnet18_out.shape == torch.Size([1, 10]), f"Expected output shape (1, 10), but got {resnet18_out.shape}"


## Question 2c: Let the training begin!

**Task:**
1. Instantiate a ResNet18 model which will classify which of the 10 classes from our Mini Imagenet dataset an image belongs to.
2. Use `AdamW` as the optimizer.
    1. [PyTorch documentation for AdamW](https://docs.pytorch.org/docs/stable/generated/torch.optim.AdamW.html)
3. Use `CrossEntropyLoss` as our criterion.
    1. [PyTorch documentation for CrossEntropyLoss](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
4. Train your ResNet18 model for 50 epochs! You should be able to reach ~90% training accuracy and ~50% validation accuracy after 50 epochs.

**Hints:**
* When you instantiate an optimizer, you also need to pass in what parameters you want this optimizer to update. You can access the parameters of any class that inherits from `nn.Module` by calling its `.parameters()` function!
* Make sure that you turn on a GPU runtime before training. It may take a few minutes to complete all 50 epochs of training. For reference, it took me (Deena) about 5-10 minutes to train my model for 50 epochs on Google Colab's T4 GPU :)

**Notes:**
* You may notice that the validation set's training accuracy lags below the training accuracy or jumps around a lot. Since we're using a toy set with only 1,000 training images and 200 validation images, that is to be expected! These metrics are an indicator that our model might be overfitting to our training set.


<!-- BEGIN QUESTION -->



In [ ]:
# TODO: Instantiate a ResNet18 model
resnet = ...

# TODO: Set the number of epochs
num_epochs = ...

# TODO: Instantiate an optimizer
optimizer = ...

# TODO: Instantiate a loss criterion
criterion = ...

# TODO: Train the model

resnet_train_accuracies, resnet_train_losses, resnet_val_accuracies, resnet_val_losses = ...

<!-- END QUESTION -->

## Question 2d: Visualize the ResNet's Training Performance

**Task:** Plot the ResNet's training loss, training accuracy, validation loss, and validation accuracy! You can use the `plot_metrics` function we defined earlier :)

<!-- BEGIN QUESTION -->



In [ ]:
plot_metrics(resnet_train_losses, resnet_val_losses, resnet_train_accuracies, resnet_val_accuracies, num_epochs, "ResNet Training Metrics")

<!-- END QUESTION -->

# Subpart 3: Transformers

<div align="center">
<img style="width: 50%;" src="https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png" />
<br>
The Transformer Architecture (source: Machine Learning Mastery)
</div>

Like ResNets, Transformers are another architecture innovation that *transformed* the world of machine learning.

Transformers were first invented for sequence-to-sequence applications, such as machine translation. In sequence-to-sequence applications, we want to take one sequence (e.g. English words) and output another sequence (e.g. its French translation).

In early seq2seq models, the entire input was compressed by an encoder into a single fixed-length vector (called the context vector). Then, this single context vector was fed into a decoder which would produce an output. [RNNs (recurrent neural networks)](https://d2l.ai/chapter_recurrent-neural-networks/index.html) improved this idea by processing data step by step, and passing information along through hidden vectors. But they struggled to capture long-range contexts and took a long time to train because they had to process each input token sequentially.

Transformers solved this problem by using *attention*. The intution behind attention is we sometimes want to dynamically *attend* to only the most important parts of the input---or of the output we have generated so far---when we want to predict what should come next in our output sequence.

## Attention

So how exactly do you determine which parts of your sequence are most relevant to each other?

Recall that we can use dot products to compute how similar 2 vectors are to each other! The higher the value of two vectors' dot product, the more 'similar' they are to each other/the closer they are to each other in vector space.

So Transformers use a mechanism called *scaled dot-product attention* to implement attention:

$$Attention(Q, K, V) = softmax(\frac{Q K^T}{\sqrt{d_k}}) V$$

* For each position in the input, the model computes three vectors: a query vector $q$, a key vector $k$, and a value vector $v$. We can stack the query, key, and value vectors into matrices $Q$, $K$, $V$, where the i-th row of $Q$ corresponds to the query vector $q_i$ of the i-th token in the sequence, etc.
* These vectors are learned linear projections of the input embeddings.
    * A helpful analogy for understanding the role that the query, key, and value vectors play is thinking about going to the library! The *query* represents what you are searching for (for instance, you might be looking for a book about matrix calculus). The *keys* for each book in the library are their titles---you use this to judge how relevant the  content in the book might be to your search. And the *value* of each library book is the actual content inside.
* To determine how much attention to pay to each part of the input, the model takes the dot product between the query vector of the current position and the key vectors of all positions in the sequence. This produces a set of scores that measure the similarity or relevance between the current position and every other position. We can represent taking dot products between all the query vectors and key vectors all at once as the matrix multiplication of $Q K^T$. The $i$-th row of the matrix product $Q K^T$ represents how similar the $i$-th query vector was to all the key vectors. Specifically, the $i, j$-th entry represents how 'similar' the $i$-th token was to the $j$-th token.
* Then, these scores are divided by the square root of the key dimension $d_k$ (to prevent their variance from growing too large), and passed through a softmax function.
* The softmax turns the scores into a probability distribution, so that all attention weights for a given token in the sequence sum to one. In other words, if you were to sum the attention weights across a single row, they would all sum up to 1. Remember that the $i$-th row corresponds to the attention weights for the $i$-th input token! So we are simply re-scaling the 'similarity scores' in each row/for each input token.
* Finally, these weights are used to compute a weighted sum of the value vectors for each position. In matrix notation, this is represented by multiplying the $softmax$ matrix by $V$.

The result is another matrix which represents the attention output for all positions in the sequence---you can think of this matrix as telling us how much each input token should "attend" to every other token!

This process allows the model to dynamically focus on the most relevant parts of the input when making predictions, and is the core building block of the Transformer architecture.

## Encoders and Decoders
Transformers consist of 2 main parts, which both use attention but in slightly different ways!

### Encoder
The encoder takes in the original input sequence and produces a vector for each input position that encodes information about that token based on the context of the entire input.

The encoder is a stack of identical layers. Each layer has:
1. **Self-Attention:** Lets every token attend to all other tokens in the input to gather global context.
2. Position-wise Feed-Forward Network (two linear layers with a nonlinearity).
3. Residual connections and LayerNorm around both sublayers to stabilize training and ease gradient flow.

### Decoder
The decoder generates outputs autoregressively (which means it generates the next token based on previous tokens it has already outputted)

The decoder is also a stack of identical layers, but each layer has three sublayers:
1. **Masked Self-Attention:** tokens from the output sequence attend to one another. The query, key, and value vectors all come from the decoder. We also apply a causal (look-ahead) mask in the decoder self-attention, which prevents a position from attending to future target tokens. During training, we pass in the entire target sequence to the decoder so we want to make sure the decoder can't "cheat" by looking ahead at future tokens when trying to predict them.
2. **Cross-Attention:** tokens from the output sequence also attend to the encoder's output, "enriching" vectors in the decoder with useful information from the encoder. The query vectors come from the decoder, and the keys/values come from the encoder.
3. Position-wise Feed-Forward Network
4. Each sublayer is wrapped with residual connections and LayerNorm.

| **Aspect** | **Encoder** | **Decoder**|
|------------|-------------|-----------|
| **Purpose** | Processes and encodes input data into rich representations (hidden states). | Generates output sequences by decoding encoded information and attending to previous outputs. |
| **Input/Output**              | Input: Full input sequence <br> Output: Contextual embeddings                                   | Input: Previously generated tokens + encoder output <br> Output: Next token(s) in output sequence            |
| **Attention Mechanism**       | **Self-attention only:** Each token attends to all **input** tokens (no masking).                | **Masked self-attention:** Each token can attend only to **previous** (and current) tokens (no future). <br> **Encoder-decoder attention:** Attends to encoder outputs to access input information. |
| **Access to Future Tokens?**  | **Yes** - sees the entire input at once; can use information from future tokens in the input.    | **No** (in self-attention) - **masking** prevents attending to future tokens during generation.              |
| **Key Components (per block)**| - LayerNorm <br> - Multi-head Self-Attention <br> - Feedforward Network (MLP) <br> - Residual connections| - LayerNorm <br> - Masked Multi-head Self-Attention <br> - Encoder-Decoder Attention <br> - Feedforward Network (MLP) <br> - Residual connections |
| **Typical Uses**              | - Encoding textual, image, or other input data <br> - Feature extraction <br> - Classifier head preparation| - Sequence generation (translation, summarization) <br> - Language modeling <br> - Autoregressive generation|

In practice, many modern machine learning architectures use either only the encoder or only the decoder part of the transformer model, depending on the task.
* Decoder-only models, such as GPT (Generative Pretrained Transformer), are very popular for tasks that involve generating text, since they predict the next token one at a time using previously generated words.
    * For text generation, a decoder-only model is sufficient because the model generates tokens one by one, each time looking at all the tokens it has already produced.
    * We "initiate" the decoding process by passing in a special start-of-sequence token.
* Encoder-only models, like BERT (Bidirectional Encoder Representations from Transformers), are widely used as embedding models to encode and understand text.
    * Encoders process the entire input sequence at once and create contextual embeddings that represetnt the relationships between all tokens in the input.
    * The contextual embeddings can be used for a variety of tasks, such as classification, clustering, or comparison.
    * This division reflects the strengths of each component: decoders are optimized for generating outputs in sequence, while encoders are designed to understand and embed input data as a whole.
    * Useful for encoding meaning, finding similarities, and making decisions about the whole input, rather than generating new content one step at a time

## Question 3a: Softmax

The softmax function converts a vector of real numbers between $[-\infty, \infty]$ into probabilities that sum up to one.

For a vector **$z$** = $[z_1, z_2, ... z_l]$, the softmax output for the $i$-th element in $z$ is given by:

$$softmax(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{k} e^{z_j}}$$

Each component $softmax(z_i) \in (0, 1)$ and all the components sum to 1.

Sometimes, you will also see the softmax output written as
$$softmax(z_i) = \frac{e^{z_i - max(z)}}{\sum_{j=1}^{k} e^{z_j - max(z)}}$$

where the maximum value of **$z$** is subtracted from each element before exponentiation. By subtracting max(z) from each input, exponentials are all kept between 0 and 1 so large numbers won't overflow into infinity and small numbers won't underflow to 0. This is sometimes used as a more numerically stable version of the softmax function for floating point arithmetic.

**Task:** Implement the softmax function (either the vanilla version, or stabilized version)

**Hints:**
* The input $x$ might be a 1-D Tensor of shape (d) or a mult-dimensional tensor of shape (B, N, N). When you apply a sum, using parameters such as `axis` to specify which dimension to sum over and `keep_dims` to avoid flattening the Tensor might be helpful!
* Some helpful PyTorch functions:
    * `torch.exp` [documentation](https://docs.pytorch.org/docs/stable/generated/torch.exp.html)
    * `torch.sum` [documentation](https://docs.pytorch.org/docs/stable/generated/torch.sum.html)

<!-- BEGIN QUESTION -->



In [ ]:
def softmax(x: torch.Tensor):
    """
    Compute the softmax of each element in x.
    """
    ...

<!-- END QUESTION -->

### Check
To check the correctness of our implementation, we can:
* Verify that the output of our softmax function has the expected dimensions (the softmax output should have the same shape as the input)
* Verify that the elements along the last dimension sum up to 1 (since softmax should transform them into a valid probability distribution)
* Compare our implementation with PyTorch's `softmax` function and check that our values are within a reasonable range of PyTorch's

In [ ]:
set_seed(SEED)

rand_1d = torch.randn(5)
print(f"Random 1D vector with shape (d): {rand_1d.shape}")
out_1d = softmax(rand_1d)

print(f"Softmax output shape: {out_1d.shape}\nExpected shape: {rand_1d.shape}\n")

# Assert that the sum of the softmax output along the last dimension is 1
assert torch.allclose(torch.sum(out_1d, dim=-1), torch.ones_like(out_1d[..., 0]), rtol=1e-5), "Sum of softmax output along last dimension is not 1"

rand_3d = torch.randn(2, 5, 5)
print(f"Random vector with shape (B, N, N): {rand_3d.shape}")
out_3d = softmax(rand_3d)
print(f"Softmax output shape: {out_3d.shape}\nExpected shape: {rand_3d.shape}\n")

# Assert that the sum of the softmax output along the last dimension is 1
assert torch.allclose(torch.sum(out_3d, dim=-1), torch.ones_like(out_3d[..., 0]), rtol=1e-5), "Sum of softmax output along last dimension is not 1"

# Compare to torch implementation
torch_softmax_1d = torch.nn.functional.softmax(rand_1d, dim=-1)
assert torch.allclose(torch_softmax_1d, out_1d, rtol=1e-5), "Softmax implementation for 1D tensor does not match torch implementation"

torch_softmax_3d = torch.nn.functional.softmax(rand_3d, dim=-1)
assert torch.allclose(torch_softmax_3d, out_3d, rtol=1e-5), "Softmax implementation for 3D tensor does not match torch implementation"

## Question 3b: Scaled Dot Product Attention

$$Attention(Q, K, V) = softmax(\frac{Q K^T}{\sqrt{d_k}}) V$$

Before we dive into implementing the scaled dot product attention mechanism, let's review what **masking** is! Masking in attention serves 2 critical purposes:
1. Causal Masking (Look-Ahead Mask) prevent future information leakage. In decoder-only models (like GPT) and the decoder portion of encoder-decoder models, we need to ensure that when predicting token at position $i$, the model can only atten to tokens at positions $\leq i$ and not future positions.
    1. Why? - During training, we have access to the entire sequence, but at inference time, we generate tokens one at a time and don't know future tokens yet
    2. Without masking, the model would "cheat" by looking at future tokens during training. This creates a mismatch between training and inference behavior

>Sentence = "The cat sat on"
>When predicting "sat", the model should only see:
>  ✓ "The"
>  ✓ "cat"
>  ✓ "sat"
>  ✗ "on"  (future token - must be masked)

2. Padding Mask - When batching sequences of different lengths, we sometimes pad shorter sequences with special tokens. We need to mask these padding positions so the model doesn't attend to meaningless values.
    1. Why? - Padding tokens have no semantic meaning, so they shouldn't influence what the model attends to. We mask [PAD] tokens so they receive zero attention weight.

> Batch of 2 sequences:
> Seq 1: "Hello world"          [length 2]
> Seq 2: "I love dinosaurs"     [length 3]
>
>After padding:
>Seq 1: "Hello world [PAD]"
>Seq 2: "I love dinosaurs"

We implement masking by setting the attention score for positions we want to mask out to $-inf$ **before** we apply softmax. After the softmax operation, these values become 0.

**Task:**
1. Implement the `scaled_dot_product_attention` function which computes the scaled dot product attention of $Q$, $K$, and $V$.
2. If `mask` is not None, use `masked_fill` to fill all values where `mask = True` with `float(-'inf')`.
3. Use the `softmax` function you implemented above to compute the softmax.

**Hints:**
* Make sure to transpose `K` befor matrix-multiplying with Q.
* Use [`masked_fill`](https://docs.pytorch.org/docs/stable/generated/torch.Tensor.masked_fill_.html#torch.Tensor.masked_fill_) to fill masked values with `float(-'inf')`.
* Don't forget to divide $Q K^T$ by the square root of the query/key vectors' dimension ($d_k$) before applying `softmax`.

**Notes:**
* As an optional debugging tool, you can use the `verbose` parameter as a flag to print out extra debugging statements (e.g. tensor shapes) inside your implementation. It is up to you whether you want to use it or not!

<!-- BEGIN QUESTION -->



In [ ]:
def scaled_dot_product_attention(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor, mask=None, verbose=False):
    """
    Q: matrix of shape (B, N, d_k)
    K: matrix of shape (B, N, d_k)
    V: matrix of shape (B, N, d_v)
    mask: boolean matrix of shape (B, N, N). Values where mask is True will be masked out

    B is the batch size
    N is the number of query/key/value vectors
    d_k is the dimensions of the query/key vectors
    d_v is the dimensions of the value vectors
    """
    assert Q.shape[-1] == K.shape[-1], "Q and K must have same d_k dimension"
    assert K.shape[-2] == V.shape[-2], "K and V must have same sequence length"


    # TODO: Implement the attention function
    ...

<!-- END QUESTION -->

### Check

Once again, let's perform a quick sanity check by checking the shape of the scaled dot product attention's output and comparing it to PyTorch's implementation.

In [ ]:
Q_tensor = torch.randn(2, 5, 10)
K_tensor = torch.randn(2, 5, 10)
V_tensor = torch.randn(2, 5, 5)

print(f"Shape of Q: {Q_tensor.shape}")
print(f"Shape of K: {K_tensor.shape}")
print(f"Shape of V: {V_tensor.shape}")

sdpa_output = scaled_dot_product_attention(Q_tensor, K_tensor, V_tensor)
print(f"Shape of output: {sdpa_output.shape}")
assert sdpa_output.shape == torch.Size([2, 5, 5]), f"Expected output shape (B, N, d_v) {Q_tensor.shape}, but got {sdpa_output.shape}"

# Test that implementation is equal to torch implementation
torch_sdp = torch.nn.functional.scaled_dot_product_attention(
    Q_tensor, K_tensor, V_tensor, attn_mask=None
)
assert torch.allclose(sdpa_output, torch_sdp, rtol=1e-5), "Scaled dot product attention implementation does not match torch implementation"

## Question 3c: Attention Head

Similarly to how we implemented our own ResidualBlocks that inherited from the `nn.Module` class, we can do the same for each part of a Transformer!

We'll start by implementing a single attention head.

**Task:**
1. Implement the __init__ method:
    1. Initialize the linear transformation matrices for queries, keys, and values.
2. Implement the `forward` method to compute attention using the `scaled_dot_product_attention` function you defined above.
    1. The `forward` method should handle both the self-attention (when encoder_output=None) and cross-attention (when encoder_output is provided) cases.
        1. Recall that during self-attention, all three of the `Q`, `K`, and `V` matrices should be generated with the input `x`.
        2. During cross-attention, `Q` should come from the `x`, and `K` and `V` should come from the encoder's output.
    2. Also, the `forward` method should apply the attention mask if provided.

**Hints:**
* Use `nn.Linear` layers to create the transformation matrices `W_q`, `W_k`, and `W_v` in the `__init__` method.
* For self-attention: use the same input `x` for queries, keys, and values.
* For cross-attention: use `x` for queries and encoder_output for keys and values.
* Call your scaled_dot_product_attention function with the computed Q, K, V matrices after you apply `W_q`, `W_k`, and `W_v` on the input `x`.

**Notes:**
* Optionally, you can save the `verbose` parameter as a member of your model (`self.verbose = verbose`). Then, you can check if `self.verbose` is `True` in your `forward` function to print extra debugging statements.

**Expected behavior:**
* Input shape: (batch_size, seq_len, d_model)
* Output shape: (batch_size, seq_len, d_k) where d_k is the dimension of the key/value vectors

<!-- BEGIN QUESTION -->



In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, d_model=512, d_k=64, verbose=False):

        # TODO: Implement the __init__ method
        # Don't forget to call super().__init__()!
        ...

    def forward(self, x, encoder_output=None, mask=None):
        """
        x is the input to use for the queries, keys, and values
        encoder_output is the output from the encoder (used for cross-attention)
        mask is the mask to use for the attention
        """


        # TODO: Implement the forward method
        ...


<!-- END QUESTION -->



In [ ]:
# Sanity check the shape of the Attention Head output
set_seed(SEED)
batch_size = 10
seq_len = 20
d_model = 512
d_k = 64

x = torch.rand((batch_size, seq_len, d_model))
print(f"Shape of x: {x.shape}")

ah = AttentionHead(d_model=d_model, d_k=d_k)

ah_out = ah(x)

print(f"Attention head output shape: {ah_out.shape}")

assert ah_out.shape == torch.Size([batch_size, seq_len, d_k]), f"Expected output shape (B, N, d_k) {batch_size, seq_len, d_k}, but got {ah_out.shape}"

## Question 3d: Multi-Head Attention

### What is Multi-Headed Attention?

Multi-headed attention runs several attention “heads” in parallel. Each head:
* Projects inputs into its own Q, K, V subspace of size `d_k = d_model / num_heads`
* Computes scaled dot-product attention independently
* Outputs a sequence of shape `(batch, seq_len, d_k)`
These head outputs are concatenated back to `(batch, seq_len, d_model)` and passed through a final linear layer to mix information across heads.

Expected shapes:
* Input: `(B, seq_len, d_model)`
* Per head: `(B, seq_len, d_k)`
* Concat: `(B, seq_len, h·d_k)` = `(B, seq_len, d_model)`
* Output projection: `(B, seq_len, d_model)`

Benefits of Multi-Headed Attention
* Multiple perspectives: Different heads can learn different relationships simultaneously (e.g., syntax vs. semantics, short-range vs. long-range).
* Subspace specialization: Learned projections let heads focus on complementary features in different representation subspaces.
* Expressivity without huge cost: Multiple small heads approximate richer interactions than a single large head, with similar compute.
* More stable learning: Lower-dimensional per-head dot products and diversity across heads can stabilize training and reduce overfitting to one pattern.

In practice, multi-head attention helps models capture richer, multi-scale dependencies across tokens, improving performance on a wide range of sequence tasks.

---

**Task:** Implement the `MultiHeadAttention` module
1. Implement the `__init__` method.
    1. Compute the dimensions of the query and key vectors in each head (`d_k`).
    2. Instantiate `num_heads` `AttentionHead` modules with `d_k`.
    3. Instantiate a linear projection that goes from `d_model` to `d_model`.
2. Implement the `forward` method.
    1. Pass the same `x`, `encoder_output` (if provided), and `mask` (if provided) through each head.
    2. Concatenate the outputs of all the heads along the last dimension.
    3. Pass the concatenated output into `W_out` to compute the final linear projection.

**Hints:**
* `d_model` refers to the the embedding/hidden size of the model’s representations. You can calculate the per-head dimension `d_k` for each head's queries/keys by dividing `d_model` by `num_heads`.
* If you are storing all your `AttentionHeads` in a plain Python list, PyTorch won't know to register their parameters later on during training! Instead, you should store them in `nn.ModuleList`.
* Each head output has shape `(batch_size, seq_len, d_k)`. What dimension should you concatenate them in to get an output with shape `(batch_size, seq_len, d_model)`?

**Expected behavior:**
* Input: `x` shape `(batch_size, seq_len, d_model)`
* Output: shape `(batch_size, seq_len, d_model)`

<!-- BEGIN QUESTION -->



In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(
        self,
        num_heads=8,
        d_model=512,  # dimension of the embeddings
        verbose=False,
    ):
        """
        Args:
            num_heads: number of attention heads
            d_model: dimension of the embeddings
            verbose: whether to print debug information
        """

        # TODO: Implement the __init__ method
        ...

    def forward(self, x, encoder_output=None, mask=None):

        # TODO: Implement the forward method
        ...


<!-- END QUESTION -->



In [ ]:
# Sanity check the shape of the Multi-Head Attention output
set_seed(SEED)
batch_size = 10
seq_len = 20
d_model = 512
num_heads = 8

x = torch.rand((batch_size, seq_len, d_model))
print(f"Shape of x: {x.shape}")

mhsa = MultiHeadAttention(num_heads=num_heads, d_model=d_model, verbose=False)

mhsa_out = mhsa(x)

print(f"Shape of Multi-Head Attention output: {mhsa_out.shape}")

assert mhsa_out.shape == torch.Size([batch_size, seq_len, d_model]), f"Expected output shape (B, N, d_model) {batch_size, seq_len, d_model}, but got {mhsa_out.shape}"

## Question 3e: Encoder Layer

The encoder stacks together self attention, a feedforward network, residual connections, and layer normalizations to take tokens from an input sequence and produce context-rich vectors.

### Feedforward Network (FFN)
The FFN is a small multi-layer perceptron applied independently at each sequence position (same weights at every position). This is just a fancy way of saying that the FFN is a linear layer! It typically follows this structure: `Linear(d_model → d_ffn) → nonlinearity (ReLU/GELU) → Linear(d_ffn → d_model)`.

`d_ffn` is the hidden/inner width of the FFN (the expansion dimension). Generally, `d_ffn` is chosen to be `4 * d_model`. After the first linear layer "expands" each vector from dimension `d_model` to `d_ffn`, the second linear layer "shrinks" the expanded vector back down to `d_model` so that it can be passed to the next `EncoderLayer` or to the Decoder.

You might ask, why would we expand the dimensions up to `d_ffn` if we're just going to shrink it back down to `d_model` again? The reason is that generally, vectors in higher-dimensions can encode more diverse types of information. We're sort of giving the FFN a chance to learn additional information about the vectors in this new, bigger feature space that it can then compress into a more useful vector even back in a feature space with only `d_model` dimensions.

### Encoder Layer Architecture
1. MultiHeadAttention with `d_model` model dimensions and `num_heads` heads
2. A residual connection to add `x` to the MultiHeadAttention's output
3. LayerNorm 1
4. A position-wise feedforward network (FFN) which consists of:
    1. A linear layer with `d_model` in_features and `d_ffn` out_features
    2. A ReLU activation
    3. A linear layer with `d_ffn` in_features and `d_model` out_features
5. A residual connection which adds the output of step 3 to the FFN's output
6. LayerNorm 2


**Task:** Implement the EncoderLayer module.
1. Implement the `__init__` method.
    1. Instantiate a `MultiHeadAttention` with `d_model` and `num_heads`.
    2. Initialize a 2-layer FFN with hidden size `d_ffn = 4 * d_model` and ReLU activations.
    3. Instantiate 2 LayerNorm modules.
2. Implement the `forward` method.
    1. Compute self-attention by passing `x` and `mask` (if not None) into the `MultiHeadAttention` block.
    2. Add a residual connection from the original input `x` to the `MultiHeadAttention`'s output.
    3. LayerNorm.
    4. Pass the normalized output through the FFN.
    5. Add a residual connection from the first LayerNorm's output to the FFN's output.
    6. LayerNorm.
    7. Return the final normalized output.


**Hints:**
* You can group all the layers of the FFN together using `nn.Sequential`: [documentation](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html).
* Don't forget to calculate what the FFN's `d_ffn` dimension should be!
* When instantiating a `LayerNorm`, you need to pass in a parameter called `normalized_shape`. `normalized_shape` specifies the size of the last dimension over which LayerNorm will compute mean/variance and learn a scale/bias parameter. What is the shape of the input to each LayerNorm layer? What is the last dimension of the shape?
* Make sure the matrices you are adding together with residual connections have matching dimensions!

**Expected behavior:**
Input: x shape `(batch_size, seq_len, d_model)`
Output: shape `(batch_size, seq_len, d_model)`

<!-- BEGIN QUESTION -->



In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model=512, num_heads=8, verbose=False):

        # TODO: Implement the __init__ method
        ...

    def forward(self, x, mask=None):

        # TODO: Implement the forward method
        # 1. Self-attention

        # 2. Residual connection

        # 3. LayerNorm

        # 4. Feedforward network

        # 5. Residual connection

        # 6. LayerNorm

        # 7. Return the output
        ...

<!-- END QUESTION -->



In [ ]:
# Sanity check the shape of the EncoderLayer output
set_seed(SEED)
batch_size = 10
seq_len = 20
d_model = 512
num_heads = 8

x = torch.rand((batch_size, seq_len, d_model)) # (B, N, d_model)
print(f"Shape of x: {x.shape}")

encoder = EncoderLayer(d_model=d_model, num_heads=num_heads, verbose=False)

encoder_out = encoder(x)

print(f"Shape of encoder layer output: {encoder_out.shape}")

assert encoder_out.shape == torch.Size([batch_size, seq_len, d_model]), f"Expected output shape (B, N, d_model) {batch_size, seq_len, d_model}, but got {encoder_out.shape}"

## Question 3f: Decoder Layer

The Decoder Layers will be very similar to the Encoder Layers, with just a few important changes:
1. The Decoder uses masked self-attention to prevent peeking at future tokens.
2. The Decoder also has a second MultiHeadAttention block for cross-attention that cross-attends to the encoder outputs.

### Decoder Layer Architecture
1. MultiHeadAttention for masked self-attention with `d_model` model dimensions and `num_heads` heads. During `forward`, we use a look-ahead mask.
2. A residual connection to add `x` to the masked self-attention's output
3. LayerNorm
4. MultiHeadAttention for cross-attention with `d_model` model dimensions and `num_heads` heads. During `forward`, we also pass in the `encoder_output` to generate the key and value vectors from.
5. A residual connection to add the output of step 3 to the cross-attention's output
6. LayerNorm
7. A position-wise feedforward network (FFN) which consists of:
    1. A linear layer with `d_model` in_features and `d_ffn` out_features
    2. A ReLU activation
    3. A linear layer with `d_ffn` in_features and `d_model` out_features
8. A residual connection which adds the output of step 6 to the FFN's output
6. LayerNorm

---

**Task:** Implement the DecoderLayer module
1. Implement `__init__`.
    1. Initialize a `MultiHeadAttention` for masked self-attention and the first `LayerNorm` layer.
    2. Initialize a `MultiHeadAttention` for cross-attention and the second `LayerNorm` layer.
    3. Initialize a 2-layer FFN with hidden size `d_ffn = 4 * d_model` and ReLU activations.
    4. Initialize a final `LayerNorm` layer.
2. Implement `forward(x, encoder_output)`.
    1. Build a causal/look-ahead mask of shape `(seq_len, seq_len)` that masks future positions
    2. Pass `x` and the look-ahead mask to the self-attention `MultiHeadAttention` block.
    3. Add a residual connection from the original input `x` to the self-attention's output.
    4. LayerNorm.
    5. Compute cross-attention using the LayerNorm's output and the `encoder_output`.
    6. Add a residual connection from the first LayerNorm's output to the cross-attention's output.
    7. LayerNorm.
    8. Compute the FFN.
    9. LayerNorm.
    10. Return the final normalized output.

**Hint:**
* How can we create an upper triangular matrix to signify what elements in the attention matrix we want to mask out during decoder self-attention? `torch.triu` ([documentation](https://docs.pytorch.org/docs/stable/generated/torch.triu.html#torch.triu)) might be helpful.

**Expected behavior:**
* Inputs: x `(B, seq_len, d_model)`, encoder_output `(B, seq_len, d_model)`
* Outputs: `(B, seq_len, d_model)`

<!-- BEGIN QUESTION -->



In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model=512, num_heads=8, verbose=False):

        # TODO: Implement the __init__ method
        ...

    def forward(self, x, encoder_output):

        # TODO: Implement the forward method

        # Create a look-ahead mask

        # 1. Masked self-attention

        # 2. Residual connection

        # 3. LayerNorm

        # 4. Cross-attention

        # 5. Residual connection

        # 6. LayerNorm

        # 7. Position-wise feed forward network

        # 8. Residual connection

        # 9. LayerNorm

        # 10. Return the final normalized output

<!-- END QUESTION -->



In [ ]:
# Sanity check the shape of the DecoderLayer output
set_seed(SEED)
batch_size = 10
seq_len = 20
d_model = 512
num_heads = 8

x = torch.rand((batch_size, seq_len, d_model)) # (B, N, d_model)
print(f"Shape of x: {x.shape}")

print("=" * 50)

encoder = EncoderLayer(d_model=d_model, num_heads=num_heads, verbose=True)
encoder_output = encoder(x)

print(f"Shape of encoder layer output: {encoder_output.shape}")

print("=" * 50)

decoder = DecoderLayer(d_model=d_model, num_heads=num_heads, verbose=True)
decoder_output = decoder(x, encoder_output)

print(f"Shape of decoder layer output: {decoder_output.shape}")

assert decoder_output.shape == torch.Size([batch_size, seq_len, d_model]), f"Expected output shape (B, N, d_model) {batch_size, seq_len, d_model}, but got {decoder_output.shape}"

## Question 3g: Positional Encodings

### What are Positional Encodings?

The last building block of a transformer we'll need is a positional encoding layer. Previous seq2seq architectures processed input tokens one-by-one. However, every word/token in the  source input sequence is passed to a transformer at once. So how does the transformer know the order of the words in the sequence?

Positional encodings add token-position information into the embeddings of the tokens. This helps the model know the ordering of the tokens within the source sequence.

In the original Transformer paper, positional encodings are implemented using $sine$ and $cosine$ waves with different frequencies depending on the position of the token and which element within that token's embedding we are looking at.

$$PE_{pos, 2i} = sin(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$$
$$PE_{pos, 2i + 1} = cos(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$$

In this equation, $pos$ refers to the position of a given token in the sequence and $i$ refers to which dimension/index of the token's embedding this value applies to.

**Task:** Implement the `PositionalEncoding` module.
1. Implement `__init__(d_model, max_len)`.
    1. Instantiate a `(max_len, d_model)` tensor of dummy values
    2. Fill even indices with `sin(pos / 10000^(2i/d_model))` and odd with `cos(...)`.
2. Implement `forward(x)`.
    1. Add the positional encodings to the the input `x` and return the sum.

**Hint:**
* Make sure that the shapes of your positional encodings either match up with the shape of `x` or are broadcastable so that adding them is a valid operation.
* Inside `forward`, make sure you move your positional encodings to the same device that `x` is on! You can get what device `x` is on my calling `x.device`.

<!-- BEGIN QUESTION -->



In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1024):
        """
        max_len = max number of tokens in the sequence
        """
        super().__init__()
        self.pos = torch.zeros((max_len, d_model))
        numerator = torch.arange(end=max_len, dtype=torch.float32).reshape(-1, 1) # shape (seq_len, 1)
        denominator = torch.pow(10000, torch.arange(start=0, end=d_model, step=2, dtype=torch.float32) / d_model).reshape(1, -1) # shape (1, d_model / 2)

        quotient = numerator / denominator # shape (seq_len, d_model). quotient[i, j] = numerator[i, 0] / denominator[0, j]
        self.pos[:, 0::2] = torch.sin(quotient) # assign result of trig to all even indices for every token positions
        self.pos[:, 1::2] = torch.cos(quotient) # assign result of trig to all odd indices for every token position

    def forward(self, x):
        """
        Input x is of shape (B, seq_len, d_embedding)
        """
        out = x + self.pos[:x.shape[1], :].to(device=x.device)
        return out

<!-- END QUESTION -->



In [ ]:
# Sanity check the shape of the PositionalEncoding output
set_seed(SEED)
pe = PositionalEncoding(512, max_len=1024)

input = torch.rand((10, 12, 512))
print(f"Shape of input: {input.shape}")

pe_output = pe(input)

print(f"Shape of positional encoding output: {pe_output.shape}")

assert pe_output.shape == input.shape, f"Expected output shape to have the same shape as the input {input.shape}, but got {pe_output.shape}"

## Question 3h: Blocks are All You Need! - TransformerEncoder

Finally, we can piece together the building blocks of our transformer!

We'll first stack an embedding layer, the `PositionalEncoding` layer we implemented, and a sequential list of the `EncoderLayers` we implemented together into a `TransformerEncoder` module.

**Task:** Implement the TransformerEncoder module.
1. Implement `__init__`.
    1. Instantiate an embedding layer using `nn.Embedding`
        1. This embedding layer will take in a token ID representing a certain token/word/ngram from our vocbulary, and output an embedding vector of size `d_model`.
    2. Instatiate a positional encoding layer using the `PositionalEncoding` class we just implemented.
    3. Instantiate `num_layers` `EncoderLayers`.
2. Implement `forward(x)`.
    1. Embed the input sequence of tokens.
    2. Add positional encodings.
    3. Sequentially pass the result through each EncoderLayer.
    4. Return the final tensor.

**Hints:**
* If you want to store all your `EncoderLayers` in a list, make sure to use `nn.ModuleList` instead of a plain Python list. Otherwise, PyTorch won't register the parameters of your `EncoderLayers` and properly update them during backpropagation!
* `nn.Embedding` takes in 2 parameters: `num_embeddings` is the number of unique tokens in your vocabulary, and `embedding_dim` is the dimension/length that each embedding vector should have. What parameters does the `__init__` method accept that correspond to the number of unique tokens/words, and the dimension of the embeddings? Note that the `embedding_dim` should be the same dimension as the vectors we pass into the `PositionalEncoding` and `EncoderLayer` layers later.

**Expected behavior:**
* Input: x shape `(batch_size, seq_len)` with integer token IDs.
* Output: shape `(batch_size, seq_len, d_model)` after all encoder layers.

<!-- BEGIN QUESTION -->



In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(
        self, vocab_size=1024, d_model=512, num_layers=6, num_heads=8, verbose=False
    ):

        # TODO: Implement the __init__ method
        ...

    def forward(self, x):

        # 1. Embed the input sequence of tokens

        # 2. Add positional encodings

        # 3. Sequentially pass the result through each EncoderLayer

        # 4. Return the final tensor
        ...

<!-- END QUESTION -->



In [ ]:
# Sanity check the shape of the TransformerEncoder output
set_seed(SEED)

transformer_encoder = TransformerEncoder(vocab_size=1024, d_model=512, num_layers=6, num_heads=8, verbose=True)

# Randomly generate a batch of token IDs with shape (B, num_tokens)
input = torch.randint(low=0, high=1024, size=(10, 100)) # (B, num_tokens)

transformer_encoder_output = transformer_encoder(input)

print(f"Shape of transformer encoder output: {transformer_encoder_output.shape}")

assert transformer_encoder_output.shape == torch.Size([10, 100, 512]), f"Expected output shape (B, num_tokens, d_model) {(10, 100, 512)}, but got {transformer_encoder_output.shape}"

## Question 3i: TransformerDecoder

Similarly, we'll stack embedding layers, `PositionalEncodings`, a sequential list of `DecoderLayers`, and lastly a linear layer to predict the next token in our vocabulary into a `TransformerDecoder` module.

The decoder turns tokens from a target sequence---or autoregressive sequence of outputs it has generated so far---into contextualized representations using masked self-attention and cross-attention to the encoder output, then projects to vocabulary logits for next-token prediction.

**Task:** Implement the TransformerDecoder module.
1. Implement `__init__`.
    1. Instantiate an embedding layer using `nn.Embedding`.
        1. This embedding layer will take in a one-hot encoded vector of size `vocab_size`, and output an embedding vector of size `d_model`.
    2. Instatiate a positional encoding layer using the `PositionalEncoding` class we just implemented.
    3. Instantiate `num_layers` `DecoderLayers`.
    4. Instantiate a final linear head to produce logits.
2. Implement `forward(x)`.
    1. Embed the input sequence of tokens.
    2. Add positional encodings.
    3. Sequentially pass the result and the `encoder_output` through each `DecoderLayer`.
    4. Apply the final linear layer to get logits over the vocabulary.
    5. Return the final tensor.

**Hints:**
* If you want to store all your `DecoderLayers` in a list, make sure to use `nn.ModuleList` instead of a plain Python list. Otherwise, PyTorch won't register the parameters of your `DecoderLayers` and properly update them during backpropagation!
* We want the output of the final linear layer to be a probability distribution across all the possible words in our vocabulary. You can think of this as a classification problem: the transformer decoder is trying to predict which class (i.e. which word) the next token should be, based on the original source sequence and the tokens it's seen so far. What should the number of `out_features` be for our linear layer?
* Don't forget to pass the `encoder_output` to each `DecoderLayer` as well!
* Your implementation of the `TransformerDecoder` should look fairly similar to your `TransformerEncoder` :)

**Expected behavior:**
* Inputs: x `(batch_size, seq_len)` (token IDs of dtype long), encoder_output `(batch_size, src_seq_len, d_model)`
* After embedding + positional encoding: `(batch_size, seq_len, d_model)`
* After stacked DecoderLayers: `(batch_size, seq_len, d_model)`
* Final output (logits): `(batch_size, seq_len, vocab_size)`

<!-- BEGIN QUESTION -->



In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size=1024, d_model=512, num_layers=6, num_heads=8, verbose=False):

        # TODO: Implement the __init__ method
        ...

    def forward(self, x, encoder_output):

        # 1. Embed the input sequence of tokens

        # 2. Add positional encodings

        # 3. Sequentially pass the result through each DecoderLayer

        # 4. Apply the linear layer to predict the next token in our vocabulary

        # 5. Return the final tensor
        ...

<!-- END QUESTION -->



In [ ]:
# Sanity check the shape of the TransformerDecoder output
set_seed(SEED)
transformer_decoder = TransformerDecoder(vocab_size=1024, d_model=512, num_layers=6, num_heads=8, verbose=True)

# Randomly generate a batch of token IDs with shape (B, num_tokens) to use as the target sequence we pass into the decoder
input = torch.randint(low=0, high=1024, size=(10, 100)) # (B, num_tokens)

# Randomly generate a batch of encoder outputs with shape (B, num_tokens, d_model)
encoder_output = torch.rand((10, 100, 512)) # (B, num_tokens, d_model)

# Pass the target sequence and encoder outputs through the decoder
transformer_decoder_output = transformer_decoder(input, encoder_output)

print(f"Shape of transformer decoder output: {transformer_decoder_output.shape}")

assert transformer_decoder_output.shape == torch.Size([10, 100, 1024]), f"Expected output shape (B, num_tokens, vocab_size) {(10, 100, 1024)}, but got {transformer_decoder_output.shape}"

## Question 3j: Transforming it all together

Putting the Encoder and Decoder together, we can create one, big, awesome `Transformer`!

**Task:** Implement the `Transformer` module.
1. Implement `__init__`.
    1. Instantiate a `TransformerEncoder` and a `TransformerDecoder`.
2. Implement `forward`.
    1. Pass `x` through the encoder.
    2. Pass `x` and the encoder's output through the decoder.
    3. Return the output of the decoder.

<!-- BEGIN QUESTION -->



In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        vocab_size=1024,
        d_model=512,
        num_encoder_layers=6,
        num_decoder_layers=6,
        num_heads=8,
        verbose=False,
    ):

        # TODO: Implement the __init__ method
        ...

    def forward(self, x):

        # TODO: Implement the forward method
        # 1. Pass `x` through the encoder

        # 2. Pass `x` and the encoder's output through the decoder

        # 3. Return the output of the decoder
        ...

<!-- END QUESTION -->



In [ ]:
# Sanity check the shape of the Transformer output
set_seed(SEED)

transformer = Transformer(
    vocab_size=1024,
    d_model=512,
    num_encoder_layers=6,
    num_decoder_layers=6,
    num_heads=8,
    verbose=True,
)

# Randomly generate a batch of token IDs with shape (B, num_tokens)
input = torch.randint(low=0, high=1024, size=(10, 100))  # (B, num_tokens)

# Pass the input through the transformer
transformer_output = transformer(input)

print(f"Shape of transformer output: {transformer_output.shape}")

assert transformer_output.shape == torch.Size([10, 100, 1024]), f"Expected output shape (B, num_tokens, vocab_size) {(10, 100, 1024)}, but got {transformer_output.shape}"

## Preparing a dataset to train the transformer

We'll load in a dataset called ["TinyStories" from Hugging Face](https://huggingface.co/datasets/roneneldan/TinyStories), which contains a list of synthetically generated short stories.

Attribution: [Eldan and Li, "TinyStories: How Small Can Language Modles Be and Still Speak Coherent English?"](https://arxiv.org/abs/2305.07759)

In [ ]:
tiny_stories = load_dataset("roneneldan/TinyStories", split='train')
tiny_stories_small = tiny_stories.shuffle(seed=SEED).select(range(1000))
print(tiny_stories_small)

Let's check out one of the short stories in this dataset!

In [ ]:
print(tiny_stories_small[0]['text'])

We define a few helper functions to pre-process the text in our dataset for transformer training.

First, we'll convert all the short stories in our text into lowercase and collect them into a list.

Then, we need to **tokenize** our text data.
**Tokenization** refers to splitting data into chunks and turning the "chunks" into numbers that we can feed into our model and embed. There's many different ways to tokenize data! For instance:
* Character-level tokenization: Represent each character or letter in the alphabet with its own token
* Byte Pair Encoding (BPE): Merge frequent character pairs to build subword units
* Word-level tokenization: Represent each word as its own token
* Phrase tokenization: Represent common phrases in a sentence as a token

In this homework, we'll keep things simple and turn each word into its own token. We need to determine the set of all unique words the comprise our vocabulary.

So we'll read each story in the `tiny_stories` dataset, collect all the paragraphs together, extract full words from each paragraph, and add them to a set.

Finally, we'll create a dictionary called `word_to_id` that maps words to a number. Later on, we can use `word_to_id` to find out which token ID each word corresponds to!

**Note**: We will also add some special tokens to our dictionary that represent the start of a sequence, pad tokens, and the end of a sequence. In training, these are often used to help delineate when phrases start/end!


In [ ]:
# Convert all the short stories to lowercase and collect them into a list
tiny_stories_text = [story['text'].lower() for story in tiny_stories_small]

# Extract all full words from text
def extract_full_words(text):
    """
    Given a string, extracts full words from the string
    Example:
        text = "The cat in the hat category"
        output = ['The', 'cat', 'in', 'the', 'hat', 'category']
    """
    pattern = r'\b\w+\b'
    return re.findall(pattern, text)

# Tokenize tiny story text - word level
paragraphs = []
for story in tiny_stories_text:
    story_paragraphs = story.split("\n\n") # split to remove new lines
    paragraphs.extend(story_paragraphs)

vocab = set()
for paragraph in paragraphs:
    full_words = extract_full_words(paragraph.lower())
    vocab.update(full_words)

word_to_id = {word: i for i, word in enumerate(vocab)}

PAD_TOKEN = '<pad>'
START_TOKEN = '<start>'
END_TOKEN = '<end>'

word_to_id[PAD_TOKEN] = len(vocab)
word_to_id[START_TOKEN] = len(vocab) + 1
word_to_id[END_TOKEN] = len(vocab) + 2

vocab.update([PAD_TOKEN, START_TOKEN, END_TOKEN])

print(vocab)
print(f"Vocab size: {len(vocab)}")

print(f"Token ID of 'park': {word_to_id['park']}")
print(f"Token ID of 'wizard': {word_to_id['wizard']}")
print(f"Token ID of 'cat': {word_to_id['cat']}")
print(f"Token ID of 'dog': {word_to_id['dog']}")


## Question 3k: Creating input and target sequences for next-token prediction

We need to create targets to train our transformer and measure how well it performs at next-token prediction. You might have noticed that our dataset earlier didn't come with any labels... but that's okay because we already have everything we need! In our case, we want to train our transformer to predict the next token. So if we pass in everything up until the $i$-th token of our input to the transformer, we want to see how close our transformer gets to predicting that the next token should be the $i+1$-th token of the story!

The key idea is that for training a transformer on next-token prediction, the inputs and the targets are the same sequence but shifted over by 1 position.

Example:
* Let's say we have the sentence: "Dinosaur nuggets taste great with ketchup."
* If we pass in the token "with" to the transformer, the *target* of its output is the actual next word in the sentence: "ketchup"
* So each training and target pair looks like the same sequence shifted by one.

**Task:** Implement `create_sequences(stories, word_to_id, max_seq_length)`.
1. Split stories into paragraphs (e.g., by double newlines) and convert them into lowercase.
2. Get a list of words from each paragraph with `extract_full_words`.
3. Add a `<start>` token and `<end>` token to the beginning and end of each paragraph.
3. For each word in the paragraph (including the `<start>` and `<end>` tokens), map them to their token ID using `word_to_id` (skip words not in the vocab)
4. Skip any paragraphs with fewer words than the `max_seq_length`.
    1. Note: We could have also chosen to pad shorter sequences, but for this homework we'll just skip over paragraphs that are too short
5. For all sequences of length `max_seq_length` in this paragraph, append it to the `input` array and append the sequence that comes 1 index position after to the `target` array
6. Convert all the inputs/targets to `torch.Tensors` with `dtype=torch.long`

**Hints:**
* Ensure returned tensors are `dtype torch.long` and have shape `(num_sequences, max_seq_length)`.
* You can reference some of the code we used to create our vocabulary dictionary to see how to split the stories into paragraphs and extract full words.

**Expected Output:**
* If the first 5 tokens in the input data look like this: [189, 42, 23, 10, 5], then the first 5 tokens in the corresponding target should look like this: [42, 23, 10, 5, 2025]

<!-- BEGIN QUESTION -->



In [ ]:
def create_sequences(stories, word_to_id, max_seq_length=20, pad_token=PAD_TOKEN, start_token=START_TOKEN, end_token=END_TOKEN):
    """
    Create input/target sequences from paragraphs

    Args:
        stories (list): List of stories to create sequences from
        word_to_id (dict): Dictionary mapping words to their token IDs
        max_seq_length (int): Maximum sequence length to create
        pad_token (str): Token for the pad token
        start_token (str): Token for the start of a sequence
        end_token (str): Token for the end of a sequence

    Returns:
        inputs (tensor): Tensor of input sequences
        targets (tensor): Tensor of target sequences
    """
    inputs = []
    targets = []

    # List of paragraphs
    paragraphs = []

    # TODO: Split the stories into paragraphs and append them to the `paragraphs` list
    ...


    # TODO: Iterate over each paragraph, extract full words, add `<start>` and `<end>` tokens, convert to IDs, and build sequences
    # Make sure to skip paragraphs that are too short to create any sequences
    ...

    # TODO: Convert the inputs and targets to tensors of type `torch.long`
    inputs = ...
    targets = ...
    return inputs, targets

# Generate sequences
max_seq_length = 20
inputs, targets = create_sequences(tiny_stories_text, word_to_id, max_seq_length)

print(f"Total sequences created: {len(inputs)}")
print(f"Each input sequence length: {len(inputs[0])}")
print(f"Each target sequence length: {len(targets[0])}")

<!-- END QUESTION -->

## Question 3l: Training and validation DataLoaders

**Task:** Split sequences and build loaders.
1. Split the inputs and targets into train/val sets with an 80/20 split.
2. Wrap the training and validation sets as `TensorDatasets`.
3. Create training and validation DataLoaders with `batch_size=32`.

**Hints:**
* `TensorDataset` is a [Dataset wrapper from PyTorch for easily creating Datasets from Tensors](https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset). It returns tuples (x[i], y[i]), and can be passed to a DataLoader just like the custom dataset we implemented earlier for our Mini ImageNet data.

**Expected behavior:**
* About 80% of sequences go to training, 20% to validation.
* A batch from the dataloaders should yield two tensors (batch_inputs, batch_targets) with shape `(32, seq_len)`. You should observe that `batch_targets` is the 1-token-left shift of `batch_inputs.`

<!-- BEGIN QUESTION -->



In [ ]:
# TODO: Split into training and validation datasets (80/20 split)
train_inputs = ...
train_targets = ...
val_inputs = ...
val_targets = ...

print(f"Number of training sequences: {len(train_inputs)}")
print(f"Number of validation sequences: {len(val_inputs)}")

# TODO: Create TensorDatasets
train_dataset = ...
val_dataset = ...

# TODO: Create DataLoaders
batch_size = 32
train_loader = ...
val_loader = ...

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

batch = next(iter(train_loader)) # Get the first batch from the training DataLoader

train_input, train_target = batch # Unpack the batch into input and target tensors
print(f"First 10 tokens of training input in first batch: {train_input[0][:10]}")
print(f"First 10 tokens of training target in first batch: {train_target[0][:10]}")

<!-- END QUESTION -->

## Question 3m: Let the transforming begin!

Using the transformer training loop provided below, train your transformer.

**Task**:
1. Instantiate a Transformer model.
2. Use `AdamW` as the optimizer.
    1. [PyTorch documentation for AdamW](https://docs.pytorch.org/docs/stable/generated/torch.optim.AdamW.html)
3. Use `CrossEntropyLoss` as our criterion.
    1. [PyTorch documentation for CrossEntropyLoss](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
4. Try training your Transformer for different numbers of epochs! Based on the training and validation loss curves we plot in the next question as well as the sample short stories generated in question 3o, pick a value for `num_epochs` that results in a model which generates the most understandable and coherent short stories.
    1. We recommend trying some values between 1 - 20.
    2. You can read more about how early stopping can actually result in "better" models: [Regularization by Early Stopping](https://www.geeksforgeeks.org/machine-learning/regularization-by-early-stopping/).

**Hints:**
* Don't forget to pass the parameters of your transformer model into the optimizer.
* Make sure that you turn on a GPU runtime before training!

In [ ]:
# Transformer training loop
def train_transformer(
    model,
    device,
    optimizer,
    criterion,
    num_epochs,
    train_dataloader,
    val_dataloader,
    vocab,
):
    # === SETUP ===
    # Move model to device
    model.to(device)

    # Lists to store metrics across epochs
    train_losses = []
    val_losses = []

    # === EPOCH LOOP ===
    for epoch in tqdm(range(num_epochs)):
        # === TRAINING PHASE ===
        model.train()  # Set model to training mode

        # Initialize metrics for this epoch
        total_train_loss = 0.0

        # === INNER LOOP (iterate over training batches) ===
        for batch_inputs, batch_targets in tqdm(train_dataloader, desc="Training"):
            # Move inputs and targets to device
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)

            # Reset the gradients
            optimizer.zero_grad()

            # Forward pass
            logits = model(batch_inputs)  # [batch_size, seq_len, vocab_size]

            # Reshape for loss calculation
            loss = criterion(logits.reshape(-1, len(vocab)), batch_targets.reshape(-1))
            # Compute loss
            loss.backward()

            # Optional: Add gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update the model parameters
            optimizer.step()

            # Accumulate the loss
            total_train_loss += loss.item()

        # === END OF INNER LOOP ===

        # Compute the average loss for this epoch
        avg_train_loss = total_train_loss / len(train_dataloader)
        train_losses.append(avg_train_loss)

        # === END OF TRAINING PHASE ===

        # === VALIDATION PHASE ===
        model.eval()  # Set model to evaluation mode
        total_val_loss = 0.0

        with torch.no_grad():
            # === INNER LOOP (iterate over validation batches) ===
            for batch_inputs, batch_targets in tqdm(val_dataloader, desc="Validation"):
                # Move inputs and targets to device
                batch_inputs = batch_inputs.to(device)
                batch_targets = batch_targets.to(device)

                # Forward pass
                logits = model(batch_inputs)

                # Compute loss
                loss = criterion(
                    logits.reshape(-1, len(vocab)), batch_targets.reshape(-1)
                )

                # Accumulate the loss
                total_val_loss += loss.item()
        # === END OF INNER LOOP ===

        # Compute the average loss for this epoch
        avg_val_loss = total_val_loss / len(val_dataloader)
        val_losses.append(avg_val_loss)

        # === END OF VALIDATION PHASE ===

        # Print epoch results
        print(f"\nEpoch {epoch + 1} Results:")
        print(f"  Training Loss: {avg_train_loss:.4f}")
        print(f"  Validation Loss: {avg_val_loss:.4f}")

    # === END OF EPOCH LOOP ===

    return train_losses, val_losses

<!-- BEGIN QUESTION -->



In [ ]:
# TODO: Instantiate a Transformer model
transformer = ...

# TODO: Set the optimizer
optimizer = ...

# TODO: Set the loss criterion
criterion = ...

# TODO: Set the number of epochs
num_epochs= ...

# TODO: Train the model
transformer_train_losses, transformer_val_losses = ...

<!-- END QUESTION -->

## Question 3n: Visualize the Transformer's Training Performance

**Task:** Plot the Transformer's training loss and validation loss. You can use the `plot_metrics` function we defined earlier :)

<!-- BEGIN QUESTION -->



In [ ]:
# TODO: Plot the training and validation loss across the epochs
...

print(f"Final Training Loss: {transformer_train_losses[-1]:.4f}")
print(f"Final Validation Loss: {transformer_val_losses[-1]:.4f}")

<!-- END QUESTION -->

## Question 3o: Transformer Inference

Our tiny transformer should now be ready to produce some of its own tiny stories! We've defined a function below that can be used to autoregressively generate new tokens from the transformer given a starting prompt. Feel free to play around with the `max_seq_len`, `max_length`, and `prompt` to see what stories you can generate with your transformer! The transformer may not be able to produce fully fluent sentences yet, but hopefully you see some coherence in the new stories it outputs.

**Note:** Because the transformer's final output is a vector of class probabilities, we can get the ID of the next token it predicts by applying an `argmax` over the final output vector. To map this token ID back into a word, we can "flip" our dictionary which mapped words to ids into one that maps ids to words. Then, we can add this newly generated word to our tiny story so far, and keep sampling the next token from the transformer, ad infinitum!

In [ ]:
def generate_sample(model, word_to_id, id_to_word, max_seq_len=20, prompt="once upon a time", max_length=50):
    """
    Autoregressively generate a sample from the transformer

    Args:
        model (Transformer): The transformer model
        word_to_id (dict): Dictionary mapping words to their token IDs
        id_to_word (dict): Dictionary mapping token IDs to words
        max_seq_len (int): Maximum sequence length used when training the transformer
        prompt (str): The starting prompt for the generated story
        max_length (int): Maximum length of the generated story

    Returns:
        str: The generated story
    """
    model.eval()
    with torch.no_grad():
        tokens = [word_to_id.get(word, 0) for word in prompt.split()]

        for _ in range(max_length):
            if len(tokens) >= max_seq_len:  # max sequence length used when training the transformer
                input_seq = torch.tensor([tokens[-1 * max_seq_len:]], dtype=torch.long, device=device)
            else:
                input_seq = torch.tensor([tokens], dtype=torch.long, device=device)

            logits = transformer(input_seq)
            next_token = torch.argmax(logits[0, -1]).item()
            tokens.append(next_token)

            if next_token == word_to_id.get(END_TOKEN, -1):
                break

        return ' '.join([id_to_word.get(token, '<unk>') for token in tokens])

id_to_word = {v: k for k, v in word_to_id.items()}


<!-- BEGIN QUESTION -->



In [ ]:
# TODO: Generate your own tiny story!
sample_text = ...
print(f"Sample generation: {sample_text}")

<!-- END QUESTION -->

# Submission Checklist

Please make sure you have completed the following steps before submitting your assignment.

## Have you...
- [ ] Saved your notebook as a PDF, ensuring that all the code cells, written questions (if applicable), and graphs are visible?
- [ ] Run the export cell in the notebook to generate a zip file containing all the code from this notebook?
- [ ] Included an acknowledgement of any GenAI assistance you used while completing this assignment, if applicable? (See the [CS 189/289 syllabus](https://eecs189.org/fa25/syllabus/#collaboration-policy-and-academic-honesty) for full details of the course's GenAI policy and for an example acknowledgement format you can use).

## Submissions
- [ ] Submitted a PDF version of your notebook to the `HW 4.1 Coding [PDF]` assignment on Gradescope?
- [ ] Submitted a zip file of your code (generated by the export cell) to the `HW 4.1 Coding [Code]` assignment on Gradescope?

Congratulations! You have completed Homework 4 Part 1!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
## Use this cell if you are running the notebook in Google Colab to install the necessary dependencies, this may take a few minutes
if IS_COLAB:
    !apt-get install -y texlive texlive-xetex pandoc


In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)